# Setup

Imports, models and pymagnitude setups happen here

In [1]:
!pip3 install pymagnitude
!wget http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude

from pymagnitude import *
vectors = Magnitude("glove.6B.300d.magnitude")

     |████████████████████████████████| 5.4MB 3.2MB/s 
  Created wheel for pymagnitude: filename=pymagnitude-0.1.120-cp36-cp36m-linux_x86_64.whl size=135918206 sha256=cbe2480ae8649b9c433e7346beb59b1539eb99cff97d8ba68e4a80a1b6a7029a
  Stored in directory: /root/.cache/pip/wheels/a2/c7/98/cb48b9db35f8d1a7827b764dc36c5515179dc116448a47c8a1
Successfully built pymagnitude
--2020-04-25 22:01:05--  http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude
Resolving magnitude.plasticity.ai (magnitude.plasticity.ai)... 52.216.136.26
Connecting to magnitude.plasticity.ai (magnitude.plasticity.ai)|52.216.136.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1384890368 (1.3G) [binary/octet-stream]
Saving to: ‘glove.6B.300d.magnitude’

glove.6B.300d.magni 100%[===================>]   1.29G  53.8MB/s    in 24s     

2020-04-25 22:01:30 (54.2 MB/s) - ‘glove.6B.300d.magnitude’ saved [1384890368/1384890368]



Add all necessary imports here

In [36]:
import numpy as np
import torch
import string
import nltk, string, os, json

from nltk import RegexpParser
from nltk.tree import Tree
import random
import ipywidgets as widgets

from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!mkdir encoder
!curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  25.3M      0  0:00:05  0:00:05 --:--:-- 29.6M


Set up models

In [4]:
# Include models.py in the directory
files.upload()
from models import InferSent
model_version = 1
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

Saving models.py to models.py


<All keys matched successfully>

In [0]:
model.set_w2v_path('drive/Shared drives/CIS 700 Project/GloVe/glove.840B.300d.txt')

In [6]:
model.build_vocab_k_words(K=100000)

Vocab size : 100000


# Functions

Similarity, JSON parser, definitions, object detection, imperatives, tokenizer

Cosine similarity

In [0]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [0]:
def find_similar_command(user_command, known_commands):

  sim1 = find_most_similar_command(user_command, known_commands)
  sim = []
  for k in known_commands:
    sim.append(cosine(model.encode(user_command)[0], model.encode(k)[0]))
  # sim = [float(i)/sum(sim) for i in sim]
  for i in range(len(sim)):
    sim[i] += sim1[i]
  idx = sim.index(max(sim))
  #print(sim)
  #print(idx)
  return known_commands[idx]

Add necessary dictionaries/lists and parsing methods here

In [0]:
def get_article(filename):
  with open(filename, "r") as f:
    s = [] # steps - list of tuples (method_number, step text)
    d = {} # descriptions - Description dictionary step text -> description
    m = [] # methods
    data_json = json.load(f)
    title = data_json['title']
    if title != None:
    # Get the steps
        if len(data_json['method/part']) > 0:
            methods = data_json['method/part']
            for method in methods:
                all_steps = [(method['name'], step['headline']) for step in method['steps']]
                s += all_steps
                m += method['name']
                all_descs = dict(zip([step['headline'] for step in method['steps']], [step['description'] for step in method['steps']]))
                d.update(all_descs)      
        else:
          # Steps could be separate from methods
          if data_json['steps']:
            for step in data_json['steps']:
              s.append(('', step['headline']))
              d[step['headline']] = step['description']
            # s = [(None, step[0]) for step in data_json['steps']]
            # d = dict(zip([step[0] for step in data_json['steps']], [step[1] for step in data_json['steps']]))
    else:
        print('Article has no title!')
    return s, d, m, title

NLTK Package Downloads

In [10]:
# download averaged_perceptron_tagger, wordnet
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Add NLTK POS-tagging and tokenizer here

In [0]:
def is_imperative(tagged_sent):
    # if the sentence is not a question...
    if tagged_sent[-1][0] != "?":
         #catches simple imperatives, e.g. "Open the pod bay doors, HAL!"
        
        if tagged_sent[0][1] == "VB" or tagged_sent[0][1] == "MD":
            return True
        elif tagged_sent[0][0].lower() == "don\'t":
          # If the first token is don't, then it is imperative as well
          return True
        # catches imperative sentences starting with words like 'please', 'you',...
        # E.g. "Dave, stop.", "Just take a stress pill and think things over."
        else:
          possible_verb = ''
          # determine which word to check, if first word is an adverb check the second one
          if tagged_sent[0][1] == "RB":
            possible_verb = tagged_sent[1][0]
          else:
            possible_verb = tagged_sent[0][0]
          # check if the first word has a verb synonym
          synonyms = [syns.name() for syns in wordnet.synsets(possible_verb)]
          # split the synonyms array into two: more related synonyms should appear earlier
          # Follow a heuristic such that if a verb synonym doesn't appear in the first half
          # then this word is likely more often used as a noun
          synonyms = synonyms[0:len(synonyms)//2]
          for synonym in synonyms:
            # if there is a verb written the same way as the first token, return true
            splitted = synonym.split('.')
            if splitted[1] == 'v':
              if splitted[0] == possible_verb.lower():
                return True
    # For the sake of our game, consider the questions as non-imperative  
    return False

# Tokenizer that I implemented from CIS421
def tokenize(text):
    tokens = []
    words = text.split(' ')
    for word in words:
        if len(word) == 0:
            continue
        w = []
        for c in word:
            if c in string.punctuation:
              if c == '\'':
                w.append(c)
              else:
                  if len(w) != 0:
                      tokens.append(''.join(w))
                      w = []
                  tokens.append(c)
            else:
                if c not in string.whitespace:
                    w.append(c)
        if len(w) != 0:
            tokens.append(''.join(w))
    return tokens

def imperative_check(text):
  # First tokenize the step headline
  tokens = tokenize(text)
  # Get the POS-tags
  tags = nltk.pos_tag(tokens)
  # Check if imperative
  imperative = is_imperative(tags)
  
  return imperative

Add similarity check functions here

In [0]:
#get vector for the given sentence
def construct_sentence_vectors(command, vectors):
  sentence_vector = np.zeros(shape=(vectors.dim,))
  for word in command.split():
    word_vector = vectors.query(word)
    sentence_vector += word_vector
  sentence_vector = sentence_vector / len(command.split())
  return sentence_vector

def construct_sentence_vector(command):
  sentence_vector = np.zeros(shape=(vectors.dim,))
  cnt = 0
  for word in command.split():
    word_vector = vectors.query(word)
    # TODO - Do something
    sentence_vector = sentence_vector + word_vector
    cnt+=1
  sentence_vector = sentence_vector/cnt
  return sentence_vector

def find_most_similar_command(user_command, known_commands):
  # TODO - Do something
  user_sent = construct_sentence_vector(user_command)
  known_sent = []
  for command in known_commands:
    known_sent.append(construct_sentence_vector(command))
  #ans = vectors.similarity(user_sent, known_sent) 
  known_sent = np.array(known_sent)
  sim = vectors.similarity(user_sent, known_sent).tolist()
  # ind = sim.index(max(sim))
  # print(sim)
  # return known_commands[ind]
  return sim


Add object identifier & 3rd person sentence formation here

In [0]:
# refered from https://stackoverflow.com/questions/41051125/turning-a-sentence-from-first-to-second-person

forms = {"am" : "are", 'i' : 'you', 'my' : 'your', 'me' : 'you', 'mine' : 'yours'} #'you' : 'I', 'your' : 'my', 'yours' : 'mine', "are" : "am"}

def translate(text):
  # print(text)
  if '"' in text:
    result = ""
    _text = text.split('"')
    l = len(_text)
    for i in range(l):
      if i%2==0:
        result += _text[i]
        for word in _text[i].split():
          # print(word)
          if word.lower() in forms: 
            result = result.replace(word, forms[word.lower()])
      else:
        s = '"' + _text[i] + '"'
        result+= s
    return result

  result = text
  for word in text.split():
    # print(word)
    if word.lower() in forms: 
      result = result.replace(word, forms[word.lower()])
      # print(result)
  # print(result)
  return result

def changePerson(text):
  result = ""
  sentences = text.split(".")
  # print(text)
  for _s in sentences:
    s = _s.strip()
    if s=="":
      continue
    tags = nltk.pos_tag(tokenize(s))
    isImp = is_imperative(tags)

    r = ""
    if isImp:
      if s[0] == ' ' or s[0] == '\n':
        r += "You " + s[1].lower() + s[2:]
      else:
        r += "You " + s[0].lower() + s[1:]
    else:
      r = s
    result += translate(r)
    result+='.\n'
  return result

def identifyObjects(text):
  tags = nltk.pos_tag(tokenize(text))
  objects = []
  for t in tags:
    if t[1]=="NN":
      objects.append(t[0])
  return objects

Description Generator

In [0]:
def get_description(title):
  DATA = ""
  tit=""
  if title.split()[0].lower()=="how" and title.split()[1].lower()=="to":
    tit += "You want "
    tit += " ".join(title.split()[1:])
  # else:
  
  objects = identifyObjects(title)
  if len(objects) == 0:
    DATA = tit
  else:
    DATA += (tit + '\n')
  # print(objects)
  if len(objects)==1:
    s = "You have a " + objects[0] + "\n"
    DATA += s
  elif len(objects)>1:
    ob = ''
    ob += ', '.join(objects)
    s = "You have " + ob + '\n'
    DATA += s
  
  print(DATA)

def get_description_title(title):
  tit=""
  title = title.lower()
  if title.split()[0].lower()=="how" and title.split()[1].lower()=="to":
    tit += "You want "
    tit += " ".join(title.split()[1:])
  print(tit)

def get_step_description_game(step, description):
  des = description[step]
  # print(des)
  if(des==""):
    return
  d = changePerson(des)
  return d

Finding similar articles for wrong choice options

In [0]:
def find_steps_similar_articles(title, n):
  vectors = Magnitude("drive/Shared drives/CIS 700 Project/finalmag.magnitude")
  v=construct_sentence_vectors(title, vectors)
  sim = vectors.most_similar(v, topn = n)
  steps = []
  for s in sim:
    f = open('drive/Shared drives/CIS 700 Project/WikihowData/js_files/' + s[0] + '.json')
    data_json = json.load(f) 
    if len(data_json['method/part']) > 0:
      methods = data_json['method/part']
      for method in methods:
          all_steps = [step['headline'] for step in method['steps']]
          steps += all_steps
    else:
      # Steps could be separate
      if data_json['steps']:
        for step in data_json['steps']:
          steps.append(step)
  return [s for s in steps if imperative_check(s)]

# Extracting Categories

Initialize the dictionary of categories



In [0]:
sub_categories = {} # initialize dict
# Studying
sub_categories['Studying'] = ['1680562.json', '1880887.json', '33391.json', '1428290.json', '8587628.json', '6004172.json', '932656.json', '933047.json', '10496288.json', '153184.json', '38565.json', '539773.json', '1631851.json', '5904158.json', '2823683.json', '260824.json', '23903.json', '8669876.json', '5457166.json', '332870.json', '133290.json', '134806.json', '464217.json', '9341291.json', '3634264.json', '635590.json', '529765.json', '5493694.json', '1147815.json', '21882.json', '1691918.json', '3333445.json', '658448.json', '220451.json', '8529840.json', '3144970.json', '1450367.json', '11615443.json', '1687874.json', '5362452.json', '476504.json', '3190797.json', '4861128.json', '697962.json', '1106709.json', '174388.json', '181624.json', '89354.json', '43338.json', '25947.json', '3987529.json', '1336876.json', '2734586.json', '9089128.json', '1107909.json', '11008991.json', '1497283.json', '92373.json', '45604.json', '1431137.json', '9705670.json', '10983507.json', '2243267.json', '1249560.json', '3304999.json', '1266736.json', '134580.json', '3664856.json', '6758699.json', '929061.json', '3920.json', '1505161.json', '268887.json', '24706.json', '6565862.json', '11635515.json', '3616972.json', '959744.json', '2444258.json', '186495.json', '4485460.json', '3621882.json', '404676.json', '217503.json', '9040836.json', '5363598.json', '72795.json', '2850572.json', '11781395.json', '2551816.json', '634948.json', '130263.json', '734329.json', '1452497.json', '9519440.json', '473531.json', '1282669.json', '5642675.json', '1356892.json', '1664016.json', '11254042.json', '153126.json', '9587949.json', '10305486.json', '1510729.json', '3863515.json', '11779532.json', '421502.json', '3123990.json', '3460954.json', '148711.json', '1753640.json', '1221970.json', '1638.json', '1405833.json', '4987579.json', '2560776.json', '1631426.json', '11307447.json', '56474.json', '67779.json', '1374920.json', '2457622.json', '1308882.json', '3627080.json', '2642081.json', '287711.json', '676605.json', '69264.json', '779738.json', '2727975.json', '4677791.json', '520876.json', '4366930.json', '8027037.json', '529991.json', '1235564.json', '124164.json', '9736409.json', '930759.json', '632849.json', '8317600.json', '1705935.json', '11506871.json', '5791019.json', '11641506.json', '11740560.json', '3311555.json', '1326990.json', '8977158.json', '1341907.json', '326687.json', '470799.json', '858534.json', '617439.json', '10218591.json', '959381.json', '1611896.json', '9104165.json', '2054.json', '9523028.json', '3064905.json', '17250.json', '3946579.json', '10557246.json', '10977434.json', '38285.json', '10221081.json', '570906.json', '9496847.json', '5069456.json', '257408.json', '3456426.json', '431554.json', '10193270.json', '1037291.json', '732116.json', '2305545.json', '595842.json', '1405131.json', '5839965.json', '336557.json', '1157834.json', '3103561.json', '9553804.json', '5423705.json', '1391189.json', '10978651.json', '42949.json', '581745.json', '1609032.json', '8869863.json', '1431229.json', '20230.json', '446691.json', '1308952.json', '516752.json', '10435763.json', '338420.json', '708617.json', '1588632.json', '672145.json', '1497060.json', '4214297.json', '1954108.json', '2624684.json', '739330.json', '453313.json', '4549603.json', '533373.json', '1082660.json', '11329596.json', '886167.json', '10122787.json', '96491.json', '649242.json', '1010025.json', '1751611.json', '137509.json', '671944.json', '9070756.json', '10767912.json', '1393566.json', '594414.json', '652750.json', '1639959.json', '242581.json', '1221711.json', '1231341.json', '1048856.json', '1450188.json', '315554.json', '131804.json', '8061820.json', '1299803.json', '9542188.json', '2729073.json', '903833.json', '256096.json', '1519417.json', '1581258.json', '1398520.json', '57907.json', '2151998.json', '68439.json', '1594774.json', '1750348.json', '710342.json', '3053787.json', '2043686.json', '8802.json', '9097662.json', '11751891.json', '1611232.json', '1439183.json', '36887.json', '8805534.json', '1519712.json', '865010.json', '386582.json', '9485965.json', '8583781.json', '310500.json', '109103.json', '151454.json', '157095.json', '663922.json', '8917446.json', '17227.json', '10533609.json', '19239.json', '11238790.json', '2284855.json', '38972.json', '10249506.json', '780629.json', '1487607.json', '2219409.json', '1387483.json', '1890756.json', '20380.json', '2799053.json', '3994105.json', '305190.json', '3079751.json', '662958.json', '595347.json', '8803384.json', '452552.json', '8586033.json', '10770383.json', '929010.json', '1773858.json', '2526983.json', '5552688.json', '943965.json', '15753.json', '7220538.json', '7048097.json', '5679562.json', '4220262.json', '10251869.json', '6066355.json', '1433194.json', '2729279.json', '2843708.json', '62156.json', '1244620.json', '5981.json', '2766442.json', '787149.json', '4947562.json', '9603337.json', '132258.json', '184545.json', '5034810.json', '11266924.json', '790264.json', '10365203.json', '672380.json', '11839.json', '9449.json', '6421788.json', '1020482.json', '752268.json', '9442920.json', '1438131.json', '2202673.json', '721802.json', '1558151.json', '1788484.json', '2952492.json', '11048844.json', '2759141.json', '6995319.json', '10968912.json', '3738046.json', '582457.json', '903838.json', '1467874.json', '223488.json', '7448913.json', '470730.json', '1280629.json', '1710766.json', '140235.json', '8776756.json', '2049412.json', '716376.json', '1226122.json', '9653728.json', '1059574.json', '8555317.json', '5779.json', '11679.json', '1243130.json', '571594.json', '10148277.json', '4744792.json', '33026.json', '1595744.json', '579350.json', '10261641.json', '109796.json', '13058.json', '544707.json', '2583294.json', '93335.json', '4024458.json', '2948380.json', '1428941.json', '739911.json', '7759635.json', '11032.json', '2213111.json', '1198942.json', '1304583.json', '5763610.json', '828938.json', '2313635.json', '4561362.json', '814857.json', '1851219.json', '1676076.json', '3331887.json', '117634.json', '8977978.json', '591871.json', '3085188.json', '7267.json', '1605299.json', '2830142.json', '1449510.json', '549123.json', '345459.json', '310946.json', '656094.json', '697964.json', '1155228.json', '306693.json', '307128.json', '1659719.json', '973632.json', '3641088.json', '1611022.json', '210219.json', '3678074.json', '2788043.json', '1667923.json', '9665745.json', '806057.json', '1328614.json', '5073379.json', '887655.json', '47853.json', '8660846.json', '722449.json', '1371682.json', '550768.json', '719963.json', '559549.json', '6053129.json', '258308.json', '20190.json', '2913604.json', '3571241.json', '6069653.json', '2600743.json', '2516912.json', '637207.json', '2265754.json', '846258.json', '6231199.json', '11579248.json', '7142018.json', '1425718.json', '1084792.json', '18814.json', '819474.json', '1406351.json', '8100611.json', '349857.json', '1570174.json', '240017.json', '5800655.json', '2901546.json', '690218.json', '1181975.json', '1794668.json', '11708712.json', '4337021.json', '1444925.json', '1199903.json', '655601.json', '492295.json', '1500372.json', '1345372.json', '1947597.json', '1229018.json', '3996503.json', '1323262.json', '129858.json', '2684143.json', '78444.json', '205827.json', '5186575.json', '451559.json', '6007870.json', '11783177.json', '527052.json', '6459776.json', '1531116.json', '1542087.json', '319398.json', '1448091.json', '2413633.json', '51457.json', '1103465.json', '1460502.json', '5287809.json', '3743303.json', '159218.json', '1336236.json', '3915185.json', '3277350.json', '11526.json', '4964219.json', '1749913.json', '11675091.json', '53885.json', '8890332.json', '9671785.json', '2850579.json', '1406668.json', '7289077.json', '411994.json', '1356860.json', '121087.json', '37443.json', '222611.json', '173831.json', '5413854.json', '732872.json', '1090529.json', '1390086.json', '1698440.json', '2507461.json', '204168.json', '10372396.json', '718502.json', '307133.json', '409595.json', '742592.json', '5040005.json', '11738347.json', '5634830.json', '221334.json', '45475.json', '645532.json', '395776.json', '1340428.json', '1343654.json', '712271.json', '10540090.json', '1693806.json', '5380889.json', '704425.json', '2930981.json', '150234.json', '1558804.json', '11015428.json', '4844543.json', '1005098.json', '1390233.json', '11635178.json', '1512085.json', '2483250.json', '212803.json', '135363.json', '28807.json', '228504.json', '1676954.json', '10122095.json', '3795116.json', '2815295.json', '3234143.json', '1344328.json', '691080.json', '1257689.json', '143822.json', '11456937.json', '3112161.json', '451693.json', '617281.json', '268766.json', '691487.json', '7634705.json', '720802.json', '3923384.json', '680783.json', '225875.json', '7952.json', '1349234.json', '767920.json', '734011.json', '434414.json', '1454944.json', '814636.json', '1044150.json', '1626736.json', '3093293.json', '386571.json', '44835.json', '1579549.json', '118889.json', '1528815.json', '226062.json', '263887.json', '2005081.json', '120730.json', '813262.json', '131967.json', '2620550.json', '8417552.json', '1415067.json', '8620352.json', '458126.json', '1442295.json', '466131.json', '9987967.json', '78426.json', '3097882.json', '5719486.json', '9762315.json', '1509408.json', '1099814.json', '1748225.json', '1538210.json', '165179.json', '122558.json', '53808.json', '87600.json', '10816485.json', '2757223.json', '1372409.json', '5747156.json', '5078887.json', '1603787.json', '3405295.json', '504603.json', '679366.json', '6585130.json', '1319669.json', '867321.json', '595778.json', '1635820.json', '8629759.json', '2137090.json', '1325090.json', '11218529.json', '1676957.json', '3229288.json', '798969.json', '5294791.json', '16184.json', '640420.json', '1373721.json', '1660401.json', '2611648.json', '1532277.json', '3033731.json', '5346806.json', '381649.json', '1533173.json', '1450011.json', '9504299.json', '662910.json', '4255634.json', '346555.json', '1752096.json', '71797.json', '5985314.json', '1914093.json', '553598.json', '68559.json', '1215841.json', '57125.json', '1393308.json', '171012.json', '1107829.json', '6298553.json', '98127.json', '1652248.json', '4680506.json', '9724332.json', '1794165.json', '9599787.json', '28644.json', '3241778.json', '5079.json', '507783.json', '1626735.json', '6053672.json', '726428.json', '8039514.json', '3660866.json', '38490.json', '1078084.json', '9578395.json', '481013.json', '7523223.json', '11629050.json', '11751683.json', '11774488.json', '10612743.json', '4281332.json', '235401.json', '1404011.json', '1371680.json', '236387.json', '437759.json', '2913094.json', '4120829.json', '1128034.json', '3127430.json', '378923.json', '8503.json', '1679895.json', '2155198.json', '2799650.json', '89242.json', '1154712.json', '5482412.json', '1375401.json', '5658319.json', '2275039.json', '2784520.json', '237241.json', '40992.json', '1332372.json', '401597.json', '2944476.json', '140997.json', '110370.json', '1151584.json', '60132.json', '729051.json', '1583535.json', '652554.json', '1623653.json', '2850868.json', '7257108.json', '896528.json', '100777.json', '860702.json', '1311500.json', '2209508.json', '1062978.json', '1893581.json', '563122.json', '714328.json', '889798.json', '1396239.json', '10824583.json', '160619.json', '391887.json', '274025.json', '4310629.json', '3795123.json', '23141.json', '9176101.json', '5490100.json', '148689.json', '4521641.json', '478363.json', '2020728.json', '2780559.json', '148666.json', '2491960.json', '1376169.json', '2913554.json', '2777359.json', '1226803.json', '339853.json', '9024694.json', '9114455.json', '1028946.json', '1952579.json', '1771305.json', '1060285.json', '748693.json', '3896527.json', '1087578.json', '1076994.json', '565262.json', '11354387.json', '5678281.json', '1377542.json', '2959068.json', '699105.json', '351458.json', '1381504.json', '2756452.json', '209001.json', '261363.json', '5361943.json', '41906.json', '482192.json', '120143.json', '1791956.json', '1469234.json', '2652509.json', '592589.json', '1158527.json', '149172.json', '1162436.json', '3472761.json', '988194.json', '282536.json', '6592683.json', '2052904.json', '1697467.json', '311207.json', '1290466.json', '2443351.json', '199905.json', '5048006.json', '11637.json', '11318996.json', '658123.json', '6938752.json', '7167342.json', '6841905.json', '5863787.json', '1954356.json', '1187677.json', '5006738.json', '409077.json', '565034.json', '1652382.json', '720932.json', '1382980.json', '67009.json', '1535205.json', '2404249.json', '1606619.json', '1431348.json', '2194890.json', '9007589.json', '593033.json', '274233.json', '5860386.json', '8069147.json', '4562520.json', '10410448.json', '2375.json', '10258295.json', '8641162.json', '2428625.json', '258833.json', '321371.json', '5546892.json', '190163.json', '4064269.json', '9995392.json', '9104090.json', '115722.json', '8147459.json', '696819.json', '2439825.json', '11534266.json', '6897726.json', '7205897.json', '261787.json', '1708416.json', '929013.json', '2889360.json', '2669881.json', '10552.json', '2209682.json', '1391226.json', '8166518.json', '1470799.json', '412748.json', '1413034.json', '2864881.json', '928985.json', '517917.json', '689236.json', '4801960.json', '8697265.json', '2850561.json', '2475054.json', '3093511.json', '11015778.json', '1099252.json', '1222733.json', '4531031.json', '1694617.json', '1404476.json', '2525343.json', '10250693.json', '39500.json', '7987898.json', '11644445.json', '1106901.json', '3554614.json', '1509063.json', '1397921.json', '3535757.json', '75917.json', '10810069.json', '391424.json', '2449183.json', '4021116.json', '4283206.json', '1454213.json', '430746.json', '1394419.json', '828274.json', '14086.json', '2161649.json', '1260757.json', '3289179.json', '10609544.json', '3140853.json', '11640996.json', '1116242.json', '303099.json', '8542097.json', '1286204.json', '10085611.json', '555070.json', '278836.json', '1937265.json', '348874.json', '490434.json', '1996747.json', '1923949.json', '511154.json', '2352571.json', '10380903.json', '3524081.json', '9219881.json', '7857925.json', '1018256.json', '929068.json', '85368.json', '730990.json', '1149075.json', '810579.json', '1337818.json', '2163281.json', '2566897.json', '805901.json', '2339433.json', '370142.json', '683718.json', '48188.json', '6725743.json', '1205133.json', '10606585.json', '4550285.json', '9230030.json', '9713499.json', '1447176.json', '433970.json', '9535673.json', '1288211.json', '1728707.json', '2930988.json', '22412.json', '9983200.json', '2523875.json', '7759826.json', '2319210.json', '1563989.json', '24084.json', '226840.json', '9536321.json', '1347576.json', '131796.json', '4599040.json', '1114171.json', '10218459.json', '705568.json', '9564345.json', '525713.json', '2751851.json', '795717.json', '11630426.json', '503733.json', '591658.json', '735865.json', '6170542.json', '7280212.json', '187224.json', '993788.json', '5648333.json', '1124343.json', '1304991.json', '103624.json', '347972.json', '1487829.json', '3109765.json', '1606809.json', '1623585.json', '30696.json', '1406143.json', '15472.json', '4076697.json', '632216.json', '1626441.json', '111208.json', '10114340.json', '699155.json', '1627315.json', '1440492.json', '313899.json', '2471303.json', '326954.json', '832111.json', '2779273.json', '4841210.json', '92107.json', '7045377.json', '19308.json', '11342389.json', '2474013.json', '683201.json', '506100.json', '9536395.json', '1721879.json', '1443210.json', '2685262.json', '7078223.json', '3795144.json', '1139785.json', '4482775.json', '3093490.json', '2344119.json', '136726.json', '800830.json', '82795.json', '610869.json', '1333512.json', '11569040.json', '434515.json', '1974840.json', '9199004.json', '2682564.json', '529383.json', '4253861.json', '29012.json', '2952547.json', '799041.json', '7937702.json', '2745849.json', '730807.json', '282585.json', '14557.json', '1513652.json', '7628666.json', '1115941.json', '1244331.json', '7266521.json', '546266.json', '1148994.json', '650172.json', '5744087.json', '721703.json', '868007.json', '415966.json', '1087524.json', '838160.json', '4259892.json', '1117655.json', '1964913.json', '1133424.json', '825990.json', '2812241.json', '4771509.json', '2609638.json', '223459.json', '1134861.json', '3371156.json', '3811.json', '5510738.json', '5734279.json', '989326.json', '6006052.json', '28213.json', '3227950.json', '8070317.json', '3617634.json', '311923.json', '195717.json', '229281.json', '104953.json', '5992593.json', '267881.json', '1421155.json', '8277570.json', '5526831.json', '246630.json', '1511394.json', '5764099.json', '123335.json', '1092336.json', '1383572.json', '929003.json', '4961052.json', '727414.json', '404321.json', '944925.json', '183817.json', '148440.json', '8423623.json', '1260454.json', '5351151.json', '550472.json', '1341769.json', '1418097.json', '7132241.json', '4637088.json', '8141342.json', '9742970.json', '1263595.json', '1568448.json', '136662.json', '1727294.json', '307571.json', '1950036.json', '30530.json', '1148805.json', '737746.json', '1971784.json', '4529240.json', '5029533.json', '1468268.json', '5887871.json', '7451721.json', '281296.json', '27308.json', '1313664.json', '1508988.json', '172584.json', '708678.json', '714122.json', '10311741.json', '442652.json', '38907.json', '11664098.json', '2729483.json', '172750.json', '8995276.json', '6951114.json', '9560192.json', '2850879.json', '151552.json', '1365892.json', '11529350.json', '150590.json', '1306329.json', '317951.json', '644096.json', '3311576.json', '1811697.json', '1517870.json', '1400491.json', '761669.json', '4257881.json', '695423.json', '10946166.json', '380268.json', '1609987.json', '5854124.json', '6126636.json', '4168378.json', '4779030.json', '3197840.json', '3785309.json', '126880.json', '1633341.json', '659870.json', '5685835.json', '647497.json', '9425479.json', '1420436.json', '7090173.json', '1254193.json', '11722216.json', '143914.json', '237246.json', '1474990.json', '1873204.json', '10042911.json', '10890322.json', '125804.json', '865897.json', '2089607.json', '842526.json', '912959.json', '42438.json', '3898010.json', '76167.json', '130981.json', '5080987.json', '1554706.json', '316268.json', '387606.json', '7782827.json', '3511453.json', '1419736.json', '9988230.json']
# Drinks
sub_categories['Drinks'] = ['3390635.json', '2106194.json', '5484239.json', '298344.json', '1547386.json', '265688.json', '1067007.json', '1674372.json', '10988760.json', '107366.json', '5715655.json', '4621539.json', '804294.json', '136587.json', '728078.json', '8542118.json', '8391460.json', '2344615.json', '5704091.json', '81508.json', '9249107.json', '676865.json', '2285394.json', '2366877.json', '215375.json', '10425580.json', '180561.json', '265098.json', '314939.json', '1327375.json', '1883229.json', '199347.json', '151195.json', '8535603.json', '2444937.json', '1098684.json', '350647.json', '337453.json', '2280909.json', '403388.json', '8917220.json', '33667.json', '10220701.json', '91752.json', '783726.json', '667096.json', '10980576.json', '1989823.json', '96081.json', '1597745.json', '11572659.json', '1890576.json', '3990761.json', '11026699.json', '6840665.json', '1135272.json', '7267559.json', '347803.json', '8229076.json', '58344.json', '475099.json', '318146.json', '45608.json', '3715856.json', '605014.json', '11026709.json', '4669625.json', '1872908.json', '329174.json', '1705962.json', '9271877.json', '1993813.json', '1762449.json', '451568.json', '8391397.json', '2324960.json', '11757476.json', '11402195.json', '2452.json', '10306896.json', '2298441.json', '11676336.json', '398142.json', '10838317.json', '423969.json', '1709405.json', '1887733.json', '34176.json', '522734.json', '769001.json', '1577496.json', '126477.json', '10435364.json', '9856237.json', '1383462.json', '1435728.json', '1066524.json', '4517150.json', '3153703.json', '545776.json', '519059.json', '1630575.json', '274352.json', '313731.json', '88749.json', '562794.json', '782546.json', '319824.json', '10418139.json', '315780.json', '131557.json', '7291894.json', '1628889.json', '2194197.json', '1371406.json', '4220195.json', '2423997.json', '284188.json', '4189986.json', '351108.json', '368770.json', '4105888.json', '24506.json', '2098203.json', '3488242.json', '52787.json', '25655.json', '131243.json', '737908.json', '721075.json', '1570559.json', '1273958.json', '2058674.json', '9757707.json', '188605.json', '90119.json', '11627319.json', '374895.json', '930406.json', '457022.json', '5710959.json', '2063939.json', '2786968.json', '33231.json', '13687.json', '93155.json', '1072126.json', '103143.json', '8520329.json', '1945972.json', '4369.json', '10563289.json', '3317669.json', '396531.json', '42108.json', '2622653.json', '26377.json', '10187263.json', '8314745.json', '114983.json', '74850.json', '8945595.json', '1038071.json', '3426109.json', '10427558.json', '2248476.json', '265246.json', '2301281.json', '2073041.json', '5827086.json', '1605352.json', '7292134.json', '21277.json', '3366897.json', '6175.json', '287234.json', '1392540.json', '993775.json', '2639547.json', '2969824.json', '9293550.json', '30295.json', '868326.json', '1993874.json', '533047.json', '775140.json', '11305775.json', '3419739.json', '53980.json', '1193973.json', '3487734.json', '758669.json', '52104.json', '3049030.json', '10431838.json', '9398802.json', '3297979.json', '10529956.json', '2253332.json', '8777777.json', '2730.json', '8387072.json', '1452809.json', '6536933.json', '519029.json', '470511.json', '24645.json', '129137.json', '2265755.json', '2388952.json', '120317.json', '7958545.json', '1222091.json', '3284354.json', '1211821.json', '8211794.json', '3716949.json', '11195578.json', '11026702.json', '31411.json', '1626311.json', '1199446.json', '6901485.json', '375884.json', '1957256.json', '11781473.json', '1128826.json', '397213.json', '4275537.json', '4921003.json', '741432.json', '5470101.json', '253046.json', '3713419.json', '1410968.json', '389919.json', '266774.json', '4862685.json', '1631856.json', '9323629.json', '10420654.json', '228027.json', '6078604.json', '3329665.json', '764438.json', '1952652.json', '7322056.json', '3574605.json', '2455989.json', '169661.json', '58569.json', '9269282.json', '387209.json', '1866736.json', '2250913.json', '3715847.json', '11620116.json', '466718.json', '1053990.json', '11718026.json', '1544870.json', '1268332.json', '1786024.json', '56977.json', '546408.json', '685610.json', '14047.json', '1147408.json', '87551.json', '855156.json', '8438315.json', '3694449.json', '11627714.json', '4951767.json', '3710444.json', '123632.json', '3141.json', '28647.json', '2881640.json', '3387051.json', '1652.json', '174028.json', '81581.json', '141467.json', '355986.json', '2255616.json', '563952.json', '5870364.json', '608321.json', '8927344.json', '1606747.json', '2282766.json', '2622387.json', '192811.json', '1063922.json', '9260848.json', '11374478.json', '6030809.json', '11453976.json', '249690.json', '4081.json', '7357.json', '317599.json', '786326.json', '14013.json', '10469419.json', '11751877.json', '351101.json', '1804349.json', '711872.json', '650861.json', '1651022.json', '11486401.json', '4103797.json', '6604931.json', '6600972.json', '1735851.json', '712659.json', '507296.json', '9077887.json', '1881322.json', '713336.json', '9365154.json', '3050.json', '1511014.json', '329202.json', '27968.json', '1654095.json', '2470774.json', '1976549.json', '4064287.json', '9459765.json', '8166647.json', '1502531.json', '409867.json', '5428547.json', '5449111.json', '10844496.json', '145310.json', '241185.json', '10425087.json', '8418104.json', '9042028.json', '358707.json', '313240.json', '2779795.json', '1226028.json', '6734683.json', '13488.json', '11751695.json', '2816320.json', '6751652.json', '2598423.json', '131092.json', '1206959.json', '1152740.json', '657582.json', '10980591.json', '9694396.json', '596318.json', '1011056.json', '1109354.json', '222700.json', '3716966.json', '1401842.json', '1395141.json', '2562779.json', '2017230.json', '2607856.json', '471114.json', '18928.json', '1115008.json', '1342122.json', '8128656.json', '448150.json', '265377.json', '133112.json', '1055489.json', '10946176.json', '1914821.json', '54138.json', '1831374.json', '128123.json', '1914837.json', '1387583.json', '10910612.json', '9293791.json', '537400.json', '1625010.json', '8884918.json', '1563760.json', '685409.json', '644383.json', '138733.json', '11212083.json', '170399.json', '11231472.json', '8074333.json', '1828207.json', '10042556.json', '1095210.json', '30982.json', '1718089.json', '10004635.json', '2282946.json', '9248617.json', '1092644.json', '58567.json', '2061907.json', '216205.json', '262847.json', '11756833.json', '38491.json', '4268624.json', '358730.json', '200410.json', '87729.json', '1722815.json', '24670.json', '662784.json', '7907976.json', '10372068.json', '1719303.json', '9230543.json', '153123.json', '168336.json', '11753830.json', '2240546.json', '686803.json', '2292376.json', '757507.json', '7219953.json', '7285125.json', '1594394.json', '2243151.json', '1140971.json', '10888523.json', '789343.json', '11749969.json', '10425073.json', '211942.json', '1091434.json', '335386.json', '10350066.json', '350639.json', '10372402.json', '767324.json', '656710.json', '4887879.json', '454226.json', '8353511.json', '1014679.json', '879118.json', '555217.json', '1281499.json', '11754983.json', '1794047.json', '1687509.json', '287089.json', '15018.json', '5032237.json', '8595292.json', '2265438.json', '1989155.json', '1393380.json', '8257919.json', '317897.json', '795500.json', '8995144.json', '2325543.json', '10935737.json', '1537706.json', '9311462.json', '8982992.json', '35713.json', '9360937.json', '8382960.json', '2597284.json', '8374619.json', '9816.json', '10417037.json', '942805.json', '1792516.json', '61949.json', '280994.json', '1215538.json', '11749714.json', '9198506.json', '9078125.json', '8685662.json', '9861.json', '620358.json', '2250795.json', '1612714.json', '9311507.json', '279762.json', '2292234.json', '8561932.json', '58297.json', '4719104.json', '1242748.json', '258802.json', '401572.json', '1807830.json', '500685.json', '2250214.json', '3795703.json', '405589.json', '135825.json', '92193.json', '95343.json', '10611958.json', '8459750.json', '115139.json', '11410885.json', '77346.json', '2116933.json', '879704.json', '908525.json', '350583.json', '2251151.json', '1558568.json', '8252412.json', '1661650.json', '5029.json', '9615618.json', '13401.json', '173090.json', '9243180.json', '1158215.json', '258907.json', '10975309.json', '6312697.json', '226174.json', '3918746.json', '397372.json', '3298345.json', '97693.json', '9245445.json', '1991513.json', '10358714.json', '82096.json', '362959.json', '10873111.json', '1650233.json', '11737624.json', '8285109.json', '10413690.json', '11646393.json', '2353262.json', '1653872.json', '259228.json', '2026419.json', '2248598.json', '1499338.json', '5797863.json', '252608.json', '3372890.json', '376170.json', '10245854.json', '3256242.json', '1464932.json', '40673.json', '1575057.json', '9434209.json', '3244044.json', '5869729.json', '2113546.json', '507675.json', '20013.json', '1612518.json', '2236935.json', '216937.json', '7566028.json', '7778881.json', '466950.json', '52910.json', '1832509.json', '1872194.json', '1452359.json', '1209916.json', '7280403.json', '1951017.json', '1480512.json', '5852875.json', '5203575.json', '10871419.json', '4241187.json', '1092992.json', '10453581.json', '10751910.json', '1532704.json', '2353151.json', '1715605.json']
# Job
sub_categories['Job Attendance'] = ['1663819.json', '1326765.json', '1204605.json', '29725.json', '7452020.json', '11680926.json', '8251646.json', '11466708.json', '426961.json', '1125891.json', '30328.json', '1291190.json', '1227459.json', '1304427.json', '251388.json', '6482124.json', '11342584.json', '7454987.json', '1962116.json', '1709440.json', '5711340.json', '94773.json', '1708372.json', '7235046.json', '1310619.json', '1077008.json', '2065240.json', '699011.json', '1544454.json', '1654694.json']
# Relationship issues
sub_categories['Relationship Issues'] = ['5901862.json', '611124.json', '1384754.json', '9104406.json', '9645232.json', '1231696.json', '7655088.json', '1261088.json', '2111939.json', '7976216.json', '139808.json', '1686674.json', '319262.json', '654943.json', '8402366.json', '2356653.json', '1192535.json', '80964.json', '18444.json', '9480.json', '14164.json', '284598.json', '2187240.json', '2985268.json', '12726.json', '397310.json', '490190.json', '416112.json', '764960.json', '257599.json', '294941.json', '768017.json', '907390.json', '7651751.json', '375362.json', '108565.json', '1572085.json', '1904582.json', '1178989.json', '117002.json', '4881864.json', '4375760.json', '288098.json', '2406528.json', '1195907.json', '3047399.json', '69767.json', '4224358.json', '1234363.json', '11691425.json', '975484.json', '10605.json', '564456.json', '139583.json', '1814634.json', '9618.json', '102391.json', '447782.json', '150818.json', '31548.json', '883099.json', '197591.json', '567216.json', '8482303.json', '640649.json', '1791323.json', '125826.json', '715475.json', '420001.json', '1037751.json', '1111782.json', '17102.json', '1660107.json', '1978749.json', '735128.json', '94658.json', '19583.json', '546480.json', '692761.json', '623644.json', '588047.json', '1982976.json', '36917.json', '94199.json', '244156.json', '945212.json', '1811569.json', '624085.json', '10354040.json', '1651355.json', '2824.json', '458729.json', '4750830.json', '6033257.json', '1559233.json', '795702.json', '9628163.json', '441684.json', '56448.json', '876733.json', '1982052.json', '964745.json', '1095399.json', '9640128.json', '2697666.json', '1313163.json', '1625048.json', '1061989.json', '10376839.json', '1950418.json', '4620318.json', '703135.json', '7875512.json', '1326128.json', '487649.json', '13255.json', '10159974.json', '354511.json', '806221.json', '10368722.json', '880831.json', '1681494.json', '230820.json', '130176.json', '375295.json', '26087.json', '8171544.json', '1115119.json', '1087469.json', '1611370.json', '287388.json', '8488705.json', '4963.json', '709021.json', '65719.json', '808049.json', '171437.json', '404191.json', '704081.json', '4877146.json', '449806.json', '1328340.json', '69687.json', '2320.json', '3316274.json', '1532583.json', '1887328.json', '798458.json', '290837.json', '8459667.json', '59266.json', '8134122.json', '21057.json', '1160130.json', '8471970.json', '32010.json', '169745.json', '18746.json', '930103.json', '5349247.json', '597766.json', '421030.json', '8780789.json', '530739.json', '55269.json', '1851259.json', '2137271.json', '82067.json', '693603.json', '2234756.json', '1947427.json', '3416382.json', '2054221.json', '2279804.json', '1603415.json', '151864.json', '7704.json', '177110.json', '8100125.json', '8656555.json', '133736.json', '293145.json', '1282005.json', '6904166.json', '874021.json', '665669.json', '4684.json', '1828264.json', '1247670.json', '2488888.json', '887232.json', '610944.json', '1263513.json', '8827564.json', '4007463.json', '9809425.json', '139617.json', '443678.json', '69558.json', '5486631.json', '1247720.json', '1512507.json', '771766.json', '3827788.json', '3893769.json', '1847286.json', '3743926.json', '4725493.json', '173185.json', '184101.json', '1320302.json', '11642882.json', '2131650.json', '4386920.json', '38243.json', '109645.json', '9072060.json', '5860738.json', '5323926.json', '817306.json', '589491.json', '1308561.json', '7508958.json', '56846.json', '1307179.json', '1522851.json', '571689.json', '794442.json', '1075182.json', '169357.json', '11676651.json', '1952939.json', '151554.json', '485883.json', '459653.json', '61276.json', '32897.json', '1569855.json', '10444297.json', '255942.json', '8461773.json', '10407289.json', '1797396.json', '22229.json', '520886.json', '2027428.json', '729986.json', '153142.json', '745035.json', '4892685.json', '138494.json', '403732.json', '8503276.json', '1721393.json', '14395.json', '61016.json', '135208.json', '331717.json', '929906.json', '2142806.json', '1546588.json', '33224.json', '7904931.json', '82585.json', '18676.json', '2675573.json', '779510.json', '2672961.json', '462827.json', '8370153.json', '566985.json', '9613836.json', '138904.json', '8654291.json', '2157690.json', '1290684.json', '8802294.json', '111273.json', '10208875.json', '34616.json', '6560654.json', '45606.json', '463285.json', '64360.json', '9000621.json', '691123.json', '17818.json', '7476127.json', '1898918.json', '4392343.json', '651104.json', '796711.json', '3608295.json', '8482697.json', '36670.json', '2185623.json', '4119934.json', '7039.json', '1174245.json', '519826.json', '5178937.json', '658597.json', '4798842.json', '57861.json', '1367271.json', '10539837.json', '13011.json', '2263098.json', '4117.json', '1116802.json', '436355.json', '945227.json', '4105891.json', '194279.json', '4732491.json', '1251488.json', '691641.json', '1058120.json', '1881325.json', '114904.json', '1107341.json', '1357392.json', '39547.json', '1602514.json', '85657.json', '1161744.json', '28839.json', '5868801.json', '167704.json', '682169.json', '1729160.json', '1291912.json', '1334868.json', '7879888.json', '41189.json', '1760436.json', '5744403.json', '3395510.json', '3128.json', '386148.json', '718117.json', '138017.json', '284578.json', '1663990.json', '1465335.json', '1243973.json', '974107.json', '5959507.json', '855611.json', '38950.json', '4772847.json', '594676.json', '42639.json', '28136.json', '398137.json', '15131.json', '310902.json', '1877.json', '1883705.json', '1668256.json', '150705.json', '26493.json', '34338.json', '648627.json', '8556334.json', '8848977.json', '9979819.json', '6090459.json', '11112879.json', '1732130.json', '3265484.json', '1621.json']
# Fashion
sub_categories['Fashion'] = ['1808118.json', '10612507.json', '817046.json', '321925.json', '31601.json', '1218144.json', '743561.json', '10091159.json', '159659.json', '2850883.json', '9671540.json', '629788.json', '1573270.json', '269015.json', '3880248.json', '1236305.json', '1958720.json', '1626029.json', '134938.json', '7977885.json', '106776.json', '10202095.json', '1120695.json', '328155.json', '4680037.json', '735129.json', '185190.json', '4144170.json', '2178284.json', '4078687.json', '211984.json', '2160175.json', '2422870.json', '2728878.json', '671787.json', '20289.json', '1181505.json', '163197.json', '10990034.json', '287802.json', '45274.json', '87628.json', '189977.json', '377559.json', '2930601.json', '10256410.json', '9627992.json', '36633.json', '10182933.json', '245822.json', '9219715.json', '566979.json', '2675608.json', '10786316.json', '893489.json', '6608644.json', '3056875.json', '10332864.json', '11763980.json', '8130351.json', '14216.json', '4753114.json', '1352279.json', '9639830.json', '611915.json', '10814940.json', '37961.json', '1539900.json', '46822.json', '2260336.json', '6456667.json', '2219350.json', '1236719.json', '5671196.json', '7654529.json', '2669903.json', '389529.json', '291502.json', '114603.json', '68659.json', '5218225.json', '78659.json', '113257.json', '33522.json', '1137720.json', '2636953.json', '11732424.json', '1508949.json', '1291855.json', '1202856.json', '4047.json', '3200508.json', '408584.json', '570707.json', '493267.json', '139540.json', '2091653.json', '403389.json', '301425.json', '447184.json', '1136081.json', '1157592.json', '8181042.json', '173323.json', '661876.json', '69951.json', '681458.json', '1998356.json', '41191.json', '320071.json', '684748.json', '125914.json', '12418.json', '113871.json', '9616542.json', '9766467.json', '5070339.json', '1682499.json', '1309532.json', '2160181.json', '3806822.json', '178369.json', '125216.json', '199385.json', '761811.json', '233456.json', '11430138.json', '88250.json', '11760328.json', '172000.json', '11764296.json', '91304.json', '9479963.json', '379717.json', '11769298.json', '718190.json', '14380.json', '1603480.json', '1486375.json', '603502.json', '355140.json', '73710.json', '1905287.json', '424004.json', '11706691.json', '229695.json', '800635.json', '196780.json', '4666190.json', '119489.json', '1135258.json', '9409860.json', '734138.json', '2281148.json', '1384637.json', '8479376.json', '5313034.json', '9522517.json', '11231892.json', '423714.json', '1399793.json', '8708372.json', '123473.json', '429769.json', '9422086.json', '648634.json', '1700642.json', '406211.json', '39958.json', '554603.json', '9976803.json', '284320.json', '180789.json', '10781621.json', '2414266.json', '1058979.json', '2765665.json', '642487.json', '2505017.json', '183865.json', '10329388.json', '8917.json', '228948.json', '634743.json', '173220.json', '48899.json', '456177.json', '3914078.json', '7234388.json', '11430451.json', '48421.json', '1461001.json', '5959.json', '9991853.json', '593066.json', '635542.json', '206698.json', '456475.json', '4056571.json', '297560.json', '247044.json', '595632.json', '5836726.json', '3192369.json', '11009094.json', '1421389.json', '2780848.json', '1386514.json', '10439434.json', '1771311.json', '5637904.json', '1371744.json', '655103.json', '11682900.json', '51841.json', '21485.json', '10980584.json', '2103846.json', '8553036.json', '2015873.json', '766813.json', '471815.json', '62663.json', '10777596.json', '163796.json', '1588570.json', '656164.json', '139252.json', '830754.json', '1351940.json', '196152.json', '6218607.json', '1243936.json', '684646.json', '9796005.json', '2130886.json', '10245684.json', '46977.json', '19966.json', '561096.json', '193686.json', '7500171.json', '8588893.json', '9780796.json', '9501759.json', '2458513.json', '688219.json', '128341.json', '38201.json', '3032174.json', '270340.json', '1354410.json', '1617600.json', '2000796.json', '15302.json', '224726.json', '1392371.json', '11783115.json', '2853726.json', '9690863.json', '1157935.json', '779383.json', '2696624.json', '9433709.json', '1074072.json', '4648185.json', '1027105.json', '594610.json', '193538.json', '11633650.json', '29628.json', '10097314.json', '2454476.json', '38594.json', '11748882.json', '1083961.json', '11379231.json', '2414803.json', '2254617.json', '2158374.json', '1527297.json', '567337.json', '679686.json', '2982511.json', '6422190.json', '3041711.json', '703479.json', '1571377.json', '188114.json', '294498.json', '483940.json', '1530753.json', '10543959.json', '1027719.json', '891193.json', '473978.json', '92025.json', '327375.json', '1333934.json', '296889.json', '70061.json', '549971.json', '25611.json', '106898.json', '11663812.json', '8391920.json', '8562047.json', '31080.json', '11757541.json', '1018364.json', '2158379.json', '8857217.json', '884851.json', '879534.json', '39058.json', '1288447.json', '9957598.json', '10163391.json', '904999.json', '283649.json', '3102905.json', '10248329.json', '2774958.json', '11611096.json', '405003.json', '1619796.json', '10147359.json', '78305.json', '34187.json', '54456.json', '310990.json', '1806482.json', '797255.json', '11715933.json', '3392363.json', '11769891.json', '10056554.json', '35407.json', '41361.json', '1134889.json', '1983640.json', '8355724.json', '1268561.json', '1218663.json', '9987337.json', '8987518.json', '5089542.json', '117380.json', '102028.json', '11366983.json', '9395710.json', '719901.json', '788483.json', '702588.json', '200955.json', '48506.json', '11642987.json', '3356167.json', '11772402.json', '6081868.json', '8195048.json', '155608.json', '59442.json', '6005.json', '8416117.json', '549696.json', '300565.json', '788728.json', '526732.json', '11674837.json', '2213132.json', '2339341.json', '9636555.json', '1554167.json', '8154916.json', '1487048.json', '90185.json', '10336930.json', '5630575.json', '5353675.json', '4738004.json', '2028220.json', '10337384.json', '5661619.json', '10357356.json', '10430903.json', '10602495.json', '1339113.json', '954396.json', '27702.json', '8580301.json', '2164483.json', '270021.json', '8687239.json', '8290812.json', '298096.json', '7296283.json', '25085.json', '788488.json', '27391.json', '2780549.json', '149708.json', '8785598.json', '1947843.json', '417218.json', '21789.json', '2248864.json', '10628658.json', '413575.json', '8334565.json', '1478455.json', '190345.json', '1525127.json', '16638.json', '613711.json', '996729.json', '990242.json', '1412024.json', '1200612.json', '220849.json', '11319027.json', '8454965.json', '423287.json', '9585472.json', '2198586.json', '1343796.json', '546773.json', '266012.json', '1401777.json', '1661083.json', '1365172.json', '375273.json', '2981208.json', '1654266.json', '290239.json', '706534.json', '2548622.json', '974582.json', '11715903.json', '618873.json', '1530888.json', '424514.json', '59192.json', '98531.json', '691339.json', '11747875.json', '7509790.json', '1979422.json', '1311120.json', '9822158.json', '2728396.json', '373667.json', '6845.json', '1079978.json', '2695625.json', '162838.json', '787967.json', '9031848.json', '1727543.json', '5600983.json', '8028379.json', '2741427.json', '2292809.json', '8043288.json', '709675.json', '1817714.json', '680375.json', '11786808.json', '27447.json', '9144165.json', '3321601.json', '1374915.json', '5247345.json', '1197962.json', '1956592.json', '337221.json', '551054.json', '599951.json', '1617472.json', '1998342.json', '896295.json', '797750.json', '2240665.json', '95661.json', '7357580.json', '2698261.json', '230887.json', '1857649.json', '158876.json', '797992.json', '1358006.json', '19539.json', '10785574.json', '758601.json', '691323.json', '9991.json', '11706163.json', '7126.json']
# Celebrities
sub_categories['Celebrities'] = ['4022027.json', '7623435.json', '185870.json', '3116492.json', '9612279.json', '9781266.json', '11250522.json', '1334670.json', '9800505.json', '2413383.json', '10193359.json', '10172566.json', '10156202.json', '1332751.json', '10998097.json', '9671560.json', '9774683.json', '3231813.json', '5318804.json', '5843284.json', '817031.json', '9859638.json', '781761.json', '4553275.json', '3829.json', '5059865.json', '9372397.json', '9972518.json', '5132822.json', '4812827.json', '4566639.json', '1549403.json', '1661235.json', '11056693.json', '8584087.json', '180608.json', '10417262.json', '11321153.json', '3464424.json', '8556265.json', '752449.json', '3448239.json', '9698968.json', '10045834.json', '9176288.json', '2958772.json', '112590.json', '6035131.json', '835916.json', '513255.json', '1869207.json', '9991665.json', '9963676.json', '11772786.json', '3201261.json', '9841296.json', '9846553.json', '376513.json', '9339225.json', '9988433.json', '1095012.json', '2204744.json', '10440255.json', '11732087.json', '1378226.json', '401404.json', '11701925.json', '8916.json', '269803.json', '194533.json', '11702077.json', '5662122.json', '4992320.json', '1225693.json', '11245199.json', '1293423.json', '9848262.json', '10180270.json', '11704044.json', '9857725.json', '728008.json', '77115.json', '11708533.json', '9763881.json', '467430.json', '9739000.json', '9696062.json', '265457.json', '62141.json', '11222114.json', '10233741.json', '4298560.json', '10389433.json', '6917567.json', '3991927.json', '10372395.json', '9814997.json', '116768.json', '295840.json', '5110583.json', '1610095.json', '1570511.json', '10052936.json', '146859.json', '6338037.json', '5959019.json', '7303909.json', '5601008.json', '1332077.json', '4468223.json', '5074596.json', '9338877.json', '1319521.json', '9991200.json', '11323605.json', '19913.json', '11575506.json', '11665800.json', '1251285.json', '103422.json', '10064188.json', '418109.json', '10138316.json', '11303207.json', '10452498.json', '3878448.json', '11701546.json', '11704314.json', '11729755.json', '1663040.json', '18271.json', '3829241.json', '11704416.json', '904311.json', '4572310.json', '32321.json', '2763100.json', '10172559.json', '1663358.json', '8458589.json', '9688848.json', '11746551.json', '11703927.json', '1728793.json', '11788378.json', '3089064.json', '11493404.json', '11329628.json', '4468086.json', '11706528.json', '10142306.json', '9746508.json', '5456000.json', '9707403.json', '122508.json', '1996491.json', '11736598.json', '9751258.json', '1758234.json', '5059774.json', '4553918.json', '5538862.json', '7406789.json', '1332676.json', '10366172.json', '2182169.json', '9798195.json', '48672.json', '11118330.json', '286089.json', '11733610.json', '3295557.json', '11702531.json', '1367250.json', '10446941.json', '1208645.json', '11725087.json']
# Dogs
sub_categories['Dogs'] = ['8207329.json', '886214.json', '5914733.json', '9230858.json', '1345461.json', '9299225.json', '1423825.json', '2182652.json', '7249692.json', '414292.json', '18242.json', '5762695.json', '10632277.json', '615130.json', '643750.json', '1081738.json', '813681.json', '4965819.json', '5656856.json', '7277797.json', '612026.json', '885253.json', '4324372.json', '1026771.json', '7056312.json', '1479661.json', '7587028.json', '7160787.json', '7455595.json', '5318865.json', '6074118.json', '2910014.json', '410443.json', '8374748.json', '7229494.json', '5767593.json', '3850255.json', '1449054.json', '6866.json', '4388920.json', '4698485.json', '299685.json', '3045783.json', '293267.json', '11788946.json', '213018.json', '1512763.json', '1289906.json', '8346496.json', '8048961.json', '8207714.json', '3642071.json', '479059.json', '4079018.json', '8271891.json', '1532984.json', '73274.json', '9648343.json', '8237064.json', '9247637.json', '89138.json', '8097934.json', '492800.json', '5591226.json', '2763518.json', '8295782.json', '1551004.json', '566133.json', '7973716.json', '7457750.json', '824966.json', '59277.json', '7675453.json', '10052545.json', '8236771.json', '1874096.json', '1221317.json', '1292171.json', '1519050.json', '413255.json', '642497.json', '2207251.json', '8098679.json', '6281613.json', '1195966.json', '11675110.json', '8332684.json', '8185796.json', '8209966.json', '7277754.json', '7193516.json', '3293479.json', '298780.json', '321266.json', '929124.json', '8313792.json', '1277557.json', '11009089.json', '47930.json', '593850.json', '1634864.json', '11238375.json', '1343556.json', '1718690.json', '7629171.json', '805427.json', '8374586.json', '1454739.json', '8205820.json', '595746.json', '9655951.json', '304558.json', '3857135.json', '419947.json', '5287681.json', '1313935.json', '18976.json', '2674832.json', '3514058.json', '9248448.json', '1185395.json', '929057.json', '8408931.json', '4800.json', '8273025.json', '7270224.json', '8277066.json', '8257829.json', '215314.json', '163894.json', '354079.json', '414288.json', '7091297.json', '22719.json', '7051530.json', '25463.json', '1394037.json', '9280858.json', '4576696.json', '707672.json', '1170052.json', '9216771.json', '3083611.json', '1576510.json', '1505747.json', '742215.json', '1344354.json', '8342066.json', '6977875.json', '1449023.json', '69272.json', '653361.json', '9310656.json', '3394640.json', '672129.json', '5168431.json', '414861.json', '4572895.json', '6781082.json', '8293135.json', '1269565.json', '4453546.json', '8336809.json', '9300139.json', '634163.json', '3787751.json', '7405610.json', '8000651.json', '45921.json', '177678.json', '2293124.json', '8073902.json', '8276837.json', '3187875.json', '1420004.json', '6201030.json', '372872.json', '6996460.json', '944361.json', '8210024.json', '8269556.json', '8010312.json', '8241495.json', '7239270.json', '11667989.json', '718154.json', '174956.json', '9897303.json', '7205935.json', '596844.json', '1207101.json', '7920103.json', '1338936.json', '5575929.json', '8318235.json', '8213062.json', '1088353.json', '7444864.json', '7004547.json', '49283.json', '7662677.json', '517207.json', '5841.json', '9316146.json', '8213218.json', '690204.json', '369287.json', '5688819.json', '1822194.json', '10106348.json', '8166208.json', '846838.json', '7288343.json', '2948889.json', '1099104.json', '1520776.json', '7363086.json', '7457618.json', '4943418.json', '613207.json', '1570168.json', '8274372.json', '10305487.json', '7311751.json', '1402924.json', '7100277.json', '127421.json', '1476366.json', '8385269.json', '2246183.json', '8355464.json', '8272926.json', '183497.json', '6974167.json', '5115086.json', '10780715.json', '2091887.json', '1942137.json', '1585407.json', '7229564.json', '5469306.json', '1394061.json', '8358153.json', '1532419.json', '444839.json', '8336536.json', '1297256.json', '7451758.json', '8237395.json', '8149176.json', '8379618.json', '7170773.json', '974083.json', '8300412.json', '8827371.json', '2383284.json', '599271.json', '283944.json', '11596265.json', '600153.json', '9294271.json', '124519.json', '82128.json', '5659096.json', '9271159.json', '4606885.json', '4560755.json', '7200536.json', '8316657.json', '4744097.json', '1808400.json', '2428.json', '2078.json', '7512625.json', '1580756.json', '1376025.json', '7432722.json', '1023196.json', '158842.json', '11247803.json', '628881.json', '1326284.json', '4124592.json', '7058634.json', '7680015.json', '3951658.json', '2958128.json', '8305630.json', '32329.json', '409570.json', '315738.json', '688100.json', '249637.json', '87648.json', '1545142.json', '2529169.json', '441027.json', '7538743.json', '9513580.json', '1353170.json', '186549.json', '1472368.json', '865991.json', '1072443.json', '7188619.json', '4637244.json', '7479183.json', '3399593.json', '3877263.json', '1566360.json', '1338039.json', '6033410.json', '7118173.json', '8269448.json', '2151218.json', '7974528.json', '72599.json', '8266528.json', '7201192.json', '8109070.json', '7610801.json', '4550013.json', '582630.json', '8272707.json', '139258.json', '5585321.json', '619190.json', '7534849.json', '11105717.json', '816457.json', '499125.json', '4728441.json', '4787604.json', '25100.json', '3765827.json', '1134679.json', '16119.json', '1469359.json', '9603469.json', '4469227.json', '5599210.json', '7114111.json', '1473241.json', '8116716.json', '377893.json', '1410082.json', '8354202.json', '7243322.json', '8352339.json', '8671155.json', '4342737.json', '767438.json', '1881297.json', '496497.json', '938020.json', '5284327.json', '10436840.json', '9453615.json', '7580095.json', '9922166.json', '2670186.json', '1505623.json', '467969.json', '5458142.json', '8027954.json', '7289319.json', '849169.json', '413014.json', '1329080.json', '7009302.json', '6486764.json', '8338027.json', '6438019.json', '1248864.json', '1849004.json', '1287923.json', '10368569.json', '7793873.json', '5660461.json', '8968409.json', '4798820.json', '1261461.json', '754437.json', '2204391.json', '6522.json', '7672987.json', '11267386.json', '1443675.json', '6917701.json', '1273630.json', '2558952.json', '28754.json', '7443666.json', '800859.json', '287541.json', '7348611.json', '24798.json', '1784999.json', '1474448.json', '1716.json', '5282651.json', '4770170.json', '265051.json', '6016796.json', '8276958.json', '791064.json', '8964860.json', '278771.json', '7608368.json', '8254716.json', '371262.json', '6904034.json', '7649672.json', '8231753.json', '5762918.json', '5516702.json', '201078.json', '8390943.json', '7279280.json', '11486815.json', '1757916.json', '3568390.json', '8298591.json', '1869010.json', '8207698.json', '180855.json', '860999.json', '11394985.json', '8338082.json', '3408827.json', '1108273.json', '1783876.json', '7134492.json', '466762.json', '5951574.json', '4612793.json', '8235053.json', '151023.json', '8357798.json', '2271074.json', '8055461.json', '5647689.json', '6084628.json', '8210825.json', '441981.json', '2654850.json', '1019413.json', '5963151.json', '6024018.json', '5365348.json', '1180810.json', '7633106.json', '8823067.json', '2594658.json', '8358586.json', '984497.json', '218058.json', '522288.json', '10564608.json', '3176923.json', '7314112.json', '5103641.json', '4860549.json', '1305062.json', '8359006.json', '1355369.json', '28056.json', '2336664.json', '1544758.json', '1144970.json', '1510934.json', '77460.json', '3871373.json', '7651833.json', '10515012.json', '7117184.json', '8272782.json', '1941614.json', '4246846.json', '7249010.json', '1798774.json', '2249181.json', '340476.json', '7457557.json', '11371863.json', '8357468.json', '716751.json', '749740.json', '1936816.json', '6540272.json', '2825244.json', '56556.json', '8319287.json', '2171294.json', '8331094.json', '2667214.json', '5991420.json', '6271494.json', '8213287.json', '8259581.json', '7871658.json', '7200069.json', '412881.json', '1257000.json', '7702671.json', '1677688.json', '547185.json', '15437.json', '7221558.json', '3432193.json', '4017379.json', '8152320.json', '6468704.json', '6192001.json', '6303308.json', '1510758.json', '1470368.json', '700781.json', '8260087.json', '2058370.json', '8352294.json', '4674725.json', '8321542.json', '9273552.json', '1484387.json', '8204094.json', '1332188.json', '1331177.json', '1858917.json', '5897593.json', '6574202.json', '7433155.json', '4304469.json', '697899.json', '9610700.json', '563272.json', '4400211.json', '7118498.json', '7732879.json', '8330304.json', '3669370.json', '8392773.json', '134528.json', '9738722.json', '8311320.json', '4793617.json', '2688008.json', '8336229.json', '205689.json', '8360019.json', '160226.json', '516671.json', '8217753.json', '883895.json', '586727.json', '30981.json', '516108.json', '272472.json', '27194.json', '8794.json', '11672811.json', '8246160.json', '8020081.json', '8389345.json', '650382.json', '636504.json', '1783114.json', '1056265.json', '4860729.json', '1383280.json', '4286334.json', '142496.json', '7128346.json', '10144258.json', '7159205.json', '53328.json', '68097.json', '7572690.json', '8388914.json', '3826401.json', '706455.json', '1857460.json', '27654.json', '7004326.json', '8355301.json', '371613.json', '8328373.json', '8325111.json', '4803744.json', '21954.json', '8369940.json', '8293578.json', '5813080.json', '5794251.json', '1848384.json', '9297206.json', '5277.json', '124483.json', '7384321.json', '16689.json', '5390217.json', '409560.json', '5320718.json', '7782589.json', '3866570.json', '8296111.json', '8283256.json', '5433133.json', '8296054.json', '1280521.json', '1618629.json', '3316425.json', '5377657.json', '4370139.json', '4237380.json', '8314226.json', '784945.json', '4692575.json', '3531433.json', '4786341.json', '1493603.json', '503223.json', '3710327.json', '8213455.json', '996130.json', '141452.json', '8295711.json', '1456430.json', '11012699.json', '8334844.json', '7354392.json', '4954355.json', '5402087.json', '1936.json', '1098288.json', '448410.json', '11330181.json', '4770595.json', '5388471.json', '7206903.json', '8275103.json', '1444335.json', '8359070.json', '6894466.json', '7099340.json', '5478730.json', '5638219.json', '8336087.json', '7641131.json', '1332367.json', '741633.json', '487279.json', '7097132.json', '6898.json', '3506756.json', '8204060.json', '8211632.json', '1656674.json', '5500481.json', '5201628.json', '319711.json', '6491068.json', '675995.json', '901900.json', '4923589.json', '8153013.json', '1532188.json', '31221.json', '8260399.json', '304898.json', '8293132.json', '409038.json', '8354727.json', '7808062.json', '9674323.json', '7628032.json', '4172585.json', '1474166.json', '1612068.json', '8314206.json', '5684580.json', '7426862.json', '723869.json', '8316112.json', '11664032.json', '8001397.json', '7829379.json', '5240373.json', '1372774.json', '7585177.json', '8318622.json', '642465.json', '556189.json', '3720924.json', '357042.json', '8850335.json', '3144845.json', '5510614.json', '7605648.json', '7985425.json', '8565260.json', '8354434.json', '7390342.json', '5924377.json', '8358552.json', '2069135.json', '7426321.json', '9592865.json', '843485.json', '7223929.json', '4653359.json', '590447.json', '7286459.json', '4322891.json', '1513657.json', '8224815.json', '311462.json', '8311839.json', '1028267.json', '50925.json', '431529.json', '6396840.json', '1046194.json', '5497799.json', '8255393.json', '1290700.json', '2060139.json', '28978.json', '7583830.json', '177595.json', '7565933.json', '8231882.json', '5493689.json', '1422442.json', '11329969.json', '7355130.json', '8310802.json', '4960981.json', '1526364.json', '450997.json', '11706540.json', '1726403.json', '5890689.json', '5984402.json', '5480158.json', '1568972.json', '1021142.json', '156509.json', '106945.json', '1722341.json', '1294279.json', '8285724.json', '1103846.json', '396948.json', '322911.json', '8234894.json', '1652233.json', '8275198.json', '139302.json', '60368.json', '1564704.json', '4026555.json', '134025.json', '5318103.json', '4359774.json', '7309260.json', '1159028.json', '2723524.json', '360666.json', '8207474.json', '9875092.json', '1353568.json', '718110.json', '8242212.json', '8309088.json', '1529381.json', '11481215.json', '7311703.json', '2300257.json', '8298045.json', '1533734.json', '7121606.json', '11225484.json', '31633.json', '5392049.json', '1180629.json', '9223207.json', '3165927.json', '2526145.json', '11456670.json', '7782640.json', '6215333.json', '7227713.json', '11674621.json', '7709988.json', '4204816.json', '154464.json', '5511092.json', '2013793.json', '1452885.json', '4862235.json', '9460605.json', '105650.json', '526027.json', '8015713.json', '1682280.json', '5360125.json', '886932.json', '11668086.json', '589777.json', '2271666.json', '510953.json', '1342494.json', '238028.json', '7027035.json', '8390238.json', '716860.json', '7245482.json', '8259576.json', '1528512.json', '8274658.json', '419693.json', '148810.json', '1221221.json', '7231405.json', '1546198.json', '4282139.json', '1302048.json', '222921.json', '333414.json', '4131561.json', '7982731.json', '7427175.json', '3485068.json', '8357446.json', '9223134.json', '2161740.json', '999585.json', '937978.json', '7959838.json', '115236.json', '25685.json', '6074599.json', '1809326.json', '677227.json', '8350550.json', '1020818.json', '5318158.json', '1268452.json', '7278299.json', '728504.json', '7579964.json', '42583.json', '5106949.json', '8050694.json', '8315727.json', '7073920.json', '8338370.json', '37745.json', '8213712.json', '8346980.json', '5197131.json', '50052.json', '8353014.json', '6405426.json', '8229496.json', '5902773.json', '5689702.json', '7536349.json', '8225289.json', '1326872.json', '2648191.json', '7572514.json', '2635675.json', '9425956.json', '4358981.json', '7567.json', '6898094.json', '208294.json', '1486642.json', '4526151.json', '8354913.json', '5141646.json', '1249319.json', '4416390.json', '5873207.json', '9220260.json', '37840.json', '3269025.json', '1339302.json', '41546.json', '232038.json', '2800325.json', '5548788.json', '8343254.json', '7517087.json', '660130.json', '6836638.json', '1374545.json', '6193583.json', '1353289.json', '1749843.json', '1420036.json', '1330174.json', '675220.json', '4791080.json', '1767784.json', '3449871.json', '7249511.json', '8313667.json', '2130626.json', '1485238.json', '8355457.json', '6065.json', '7055756.json', '62288.json', '987919.json', '5521982.json', '1852476.json', '6206769.json', '8211996.json', '6928694.json', '193559.json', '1475892.json', '9236064.json', '410406.json', '1357458.json', '4241285.json', '1125000.json', '136192.json', '2837.json', '723298.json', '8333004.json', '8237325.json', '1339745.json', '1555097.json', '4404791.json', '1776585.json', '7426856.json', '3026178.json', '1792454.json', '3490267.json', '5112161.json', '131601.json', '7193753.json', '25217.json', '8318303.json', '1251342.json', '1238774.json', '8317019.json', '7360.json', '5722466.json', '8400364.json', '1445967.json', '6174906.json', '5348911.json', '210220.json', '763652.json', '7978348.json', '4876294.json', '11336104.json', '8292469.json', '5303390.json', '471913.json', '4153242.json', '16648.json', '8153709.json', '2331593.json', '2412132.json', '4954896.json', '887945.json', '138877.json', '28258.json', '7277679.json', '7756582.json', '7201155.json', '7654833.json', '8358931.json', '1520545.json', '528624.json', '1270425.json', '9586500.json', '1097450.json', '7328064.json', '619710.json', '1627778.json', '8056184.json', '218942.json', '3921289.json', '9639897.json', '1651812.json', '290931.json', '5596509.json', '413991.json', '8233976.json', '1158337.json', '958536.json', '7503790.json', '8258184.json', '8260397.json', '1404769.json', '1047973.json', '596661.json', '209703.json', '8318378.json', '1288579.json', '7633220.json', '7355145.json', '10177096.json', '2064226.json', '368430.json', '9991294.json', '3816236.json', '3959145.json', '7073649.json', '1348554.json', '8209125.json', '6222309.json', '7510020.json', '491120.json', '7858347.json', '8276849.json', '62952.json', '7729646.json', '1957327.json', '8260708.json', '5944061.json', '8389809.json', '582317.json', '7386406.json', '104685.json', '1361619.json', '5516347.json', '8322137.json', '9688984.json', '2115836.json', '5321644.json', '7092386.json', '11013205.json', '8029407.json', '1344384.json', '8212876.json', '7626805.json', '7231107.json', '8225093.json', '4984110.json', '39107.json', '3221601.json', '390327.json', '9605825.json', '1311761.json', '11764000.json', '19677.json', '2288887.json', '1391267.json', '5838258.json', '2231491.json', '8234285.json', '8211124.json', '16613.json', '5501139.json', '26925.json', '4647174.json', '7242612.json', '54319.json', '128178.json', '5667586.json', '3480912.json', '1351981.json', '6843365.json', '93681.json', '8362713.json', '10578540.json', '5941.json', '7472440.json', '1375375.json', '52170.json', '513344.json', '5360734.json', '8271898.json', '1732706.json', '30473.json', '1531163.json', '7230628.json', '3038916.json', '6974837.json', '162828.json', '691387.json', '6885886.json', '6965549.json', '5360763.json', '7227694.json', '8309089.json', '286685.json', '7481.json', '5115391.json', '134248.json', '29185.json', '11769825.json', '6012596.json', '735383.json', '8101632.json', '8391003.json', '1214729.json', '44289.json', '565645.json', '7632344.json', '4197894.json', '7229366.json', '2697537.json', '6095217.json', '23160.json', '8117498.json', '2055465.json', '31018.json', '4134060.json', '4707520.json', '10592925.json', '182152.json', '11338.json', '1580953.json', '5988321.json', '7756442.json', '8357913.json', '417097.json', '5540150.json', '1352918.json', '8317198.json', '399422.json', '8386325.json', '20878.json', '45581.json', '1488709.json', '516337.json', '221385.json', '9928.json', '3362921.json', '8148713.json', '1082182.json', '1388889.json', '8335817.json', '8285796.json', '1435526.json', '676467.json', '6098300.json', '4068060.json', '2053451.json', '245206.json', '4304601.json', '7556221.json', '2629938.json', '5063541.json', '1557428.json', '262749.json', '930080.json', '7685890.json', '118013.json', '549737.json', '8185690.json', '1421089.json', '6082035.json', '297174.json', '7662269.json', '6970828.json', '21005.json', '7289789.json', '4791045.json', '9273805.json', '4973275.json', '7536748.json', '1146545.json', '5432160.json', '6904211.json', '1829597.json', '3423440.json', '9606886.json', '8206039.json', '519555.json', '1719975.json', '300368.json', '4295571.json', '74065.json', '4757026.json', '795890.json', '8352678.json', '7905556.json', '7193312.json', '7580215.json', '61271.json', '5009809.json', '45391.json', '163475.json', '6953333.json', '8316822.json', '8258443.json', '1475894.json', '8379569.json', '113059.json', '9256360.json', '2243926.json', '5948419.json', '9298965.json', '647603.json', '8316208.json', '4792673.json', '674401.json', '7960933.json', '754506.json', '1487673.json', '6042788.json', '1181614.json', '7151810.json', '1431586.json', '8275171.json', '1350641.json', '788543.json', '8421275.json', '7355322.json', '8317205.json', '1000266.json', '8254010.json', '587128.json', '9666531.json', '2261477.json', '7402987.json', '113346.json', '207700.json', '345538.json', '6096187.json', '8358652.json', '6904231.json', '7985027.json', '8233992.json', '7169300.json', '8150010.json', '8318627.json', '11598184.json', '451897.json', '6443427.json', '5671642.json', '2595260.json', '1455813.json', '297178.json', '4221733.json', '88662.json', '4708914.json', '265604.json', '5688836.json', '4863863.json', '412831.json', '1520535.json', '4062271.json', '4618027.json', '6485556.json', '579067.json', '9268947.json', '1393436.json', '1813170.json', '4791823.json', '8207223.json', '8211318.json', '462637.json', '879250.json', '1194586.json', '7267760.json', '4988071.json', '8317287.json', '7099758.json', '9547184.json', '3214661.json', '8211174.json', '3283267.json', '1312019.json', '3328.json', '5859074.json', '8197859.json', '7311292.json', '608999.json', '5749066.json', '7860684.json', '7702395.json', '9545906.json', '1475436.json', '769604.json', '576412.json', '318257.json', '1352650.json', '8358957.json', '968762.json', '27333.json', '6033433.json', '9473594.json', '8214622.json', '5185346.json', '7561867.json', '58147.json', '1803023.json', '1465877.json', '7609746.json', '4517407.json', '8297159.json', '4869716.json', '5264242.json', '1681813.json', '8298613.json', '4641233.json', '476264.json', '8004797.json', '7603576.json', '4021254.json', '9462384.json', '2138358.json', '1155098.json', '6347.json', '8311201.json', '405571.json', '7210975.json', '2725926.json', '1378994.json', '761389.json', '7983189.json', '1267846.json', '10860855.json', '608961.json', '3487910.json', '7009176.json', '22034.json', '1103528.json', '165555.json', '1752.json', '1665983.json', '8717515.json', '3346233.json', '446447.json', '7354621.json', '781889.json', '1144277.json', '3506932.json', '1779953.json', '589377.json', '4943436.json', '433681.json', '8210949.json', '311124.json', '9771441.json', '4274569.json', '6540406.json', '1763133.json', '8304605.json', '6601077.json', '1487408.json', '103605.json', '1651662.json', '6904660.json', '706735.json', '1060523.json', '11469499.json', '4898369.json', '501451.json', '8342032.json', '7221253.json', '4493974.json', '426440.json', '5475219.json', '15900.json', '626018.json', '1052014.json', '9240153.json', '7702750.json', '701009.json', '681114.json', '3996819.json', '212000.json', '174283.json', '8191762.json', '1329333.json', '735444.json', '1682317.json', '8259674.json', '5490062.json', '5217081.json', '11022437.json', '5512841.json', '43008.json', '8210498.json', '8169917.json', '705271.json', '6599417.json', '8335908.json', '9248758.json', '427322.json', '8010367.json', '3697943.json', '8358409.json', '7904868.json', '8859258.json', '8255509.json', '1360239.json', '6352251.json', '268406.json', '134281.json', '405879.json', '4996218.json', '8234558.json', '1377544.json', '4549031.json', '7003977.json', '586759.json', '11720641.json', '6022278.json', '1367728.json', '288605.json', '1435730.json', '5340424.json', '8317273.json', '8069839.json', '9295878.json', '8316198.json', '6915285.json', '22468.json', '1446967.json', '5005647.json', '8335178.json', '57354.json', '84230.json', '5913722.json', '8276947.json', '3316860.json', '4726471.json', '601998.json', '8282984.json', '7313575.json', '8381033.json', '8318082.json', '4025962.json', '6659941.json', '953215.json', '6483030.json', '1196719.json', '5213467.json', '4966.json', '8318295.json', '1719775.json', '11704148.json', '5526391.json', '8023743.json', '181918.json', '7609399.json', '2301334.json', '8325553.json', '11781477.json', '8321140.json', '1454921.json', '7483234.json', '4394243.json', '8860363.json', '5695819.json', '7927277.json', '1590518.json', '449911.json', '1875112.json', '780484.json', '724824.json', '420561.json', '1639633.json', '7851005.json', '8203083.json', '42584.json', '1250742.json', '8456129.json', '305556.json', '7249486.json', '1170876.json', '1508880.json', '4585633.json', '7748774.json', '9251826.json', '8336851.json', '4970767.json', '8274940.json', '657045.json', '594174.json', '1920512.json', '8297968.json', '8301364.json', '8360227.json', '203563.json', '7278158.json', '650295.json', '1856019.json']
# University
sub_categories['University'] = ['2723296.json', '8533558.json', '9466777.json', '2611713.json', '11692679.json', '10346744.json', '11062965.json', '9814.json', '285937.json', '521656.json', '9053353.json', '89682.json', '10560856.json', '1328147.json', '2146636.json', '4948275.json', '1419308.json', '707193.json', '1118982.json', '5553046.json', '10801718.json', '147450.json', '6658739.json', '3228281.json', '49627.json', '1534738.json', '33647.json', '9485323.json', '2187169.json', '1405254.json', '69287.json', '2669723.json', '372891.json', '2197647.json', '2519618.json', '1674145.json', '2438745.json', '5718196.json', '2091310.json', '11763914.json', '1683084.json', '69024.json', '5962156.json', '3637147.json', '11254516.json', '11051073.json', '9657583.json', '316573.json', '8455337.json', '1735906.json', '439667.json', '1394575.json', '1424704.json', '4489547.json', '1256162.json', '11488732.json', '9925180.json', '1163691.json', '9560164.json', '1902682.json', '9124205.json', '2672876.json', '7038311.json', '3678357.json', '1064060.json', '56462.json', '1778414.json', '442344.json', '9594871.json', '9351862.json', '9226017.json', '11023730.json', '6364878.json', '9416951.json', '10038.json', '679375.json', '9545929.json', '2415295.json', '415621.json', '1935030.json', '1328035.json', '1709106.json', '4639190.json', '9627881.json', '4851897.json', '10175666.json', '1409796.json', '1057717.json', '5149802.json', '1136424.json', '1013408.json', '9339644.json', '11048707.json', '6362539.json', '139852.json', '102201.json', '6361895.json', '104947.json', '4727901.json', '237264.json', '2785178.json', '7954.json', '6579.json', '1519163.json', '2601572.json', '11296665.json', '1664144.json', '1120437.json', '9199820.json', '4746527.json', '11755295.json', '4567663.json', '1094239.json', '1555953.json', '683127.json', '1146287.json', '8212686.json', '1108891.json', '3373919.json', '11794732.json', '1169252.json', '1169317.json', '1149575.json', '417981.json', '2039166.json', '1412752.json', '2850882.json', '1213898.json', '10313436.json', '7421006.json', '858851.json', '2850569.json', '10843784.json', '277749.json', '1526339.json', '1359678.json', '11371188.json', '4180334.json', '1391296.json', '893555.json', '10908267.json', '1371216.json', '2519398.json', '1151578.json', '9439788.json', '514688.json', '1441993.json', '13404.json', '4957523.json', '1479780.json', '2780853.json', '94014.json', '2411707.json', '3311343.json', '1220824.json', '7824198.json', '11003590.json', '4444641.json', '46223.json', '871672.json', '4415567.json', '11275758.json', '1470626.json', '3946905.json', '18209.json', '1117647.json', '9496354.json', '1170795.json', '122171.json', '9569314.json', '7019870.json', '1218764.json', '9693723.json', '1170668.json', '406460.json', '3303651.json', '10346518.json', '1371765.json', '10358013.json', '16542.json', '2253943.json', '1730387.json', '9294535.json', '836402.json', '2319189.json', '11399620.json', '2765203.json', '1216982.json', '7999016.json', '9398459.json', '1532189.json', '113354.json', '9636628.json', '5527570.json', '4983603.json', '1850982.json', '7112037.json', '5413988.json', '10371807.json', '1470407.json', '1187863.json', '1530481.json', '373654.json', '1397312.json', '46444.json', '9431429.json', '181136.json', '61672.json', '1192948.json', '9605304.json', '1509241.json', '355731.json', '134050.json', '201062.json', '9606986.json', '1435735.json', '9288166.json', '10783923.json', '4157982.json', '372950.json', '6372027.json', '9501538.json', '11701764.json', '246402.json', '9072074.json', '1518606.json', '8087461.json', '5472193.json', '6199732.json', '3609434.json', '5337422.json', '138771.json', '3311334.json', '1108880.json', '1914731.json', '10532875.json', '698412.json', '5297219.json', '10355840.json', '204534.json', '2974387.json', '1122302.json', '906772.json', '11759208.json', '1220002.json', '5904831.json', '577549.json', '1536094.json', '9928957.json', '9508626.json', '7933436.json', '4570460.json', '77162.json', '186467.json', '2293545.json', '11275190.json', '16076.json', '11701948.json', '6765565.json', '8018587.json', '5659618.json', '1203426.json', '9996085.json', '1345431.json', '679646.json', '47080.json', '5516963.json', '10355641.json', '27506.json', '56853.json', '841334.json', '1156038.json', '1314193.json', '8583532.json', '5212454.json', '9666349.json', '5104644.json', '61829.json', '56465.json', '2670118.json', '10402744.json', '5395737.json', '42125.json', '30021.json', '771572.json', '9875746.json', '1124389.json', '161617.json', '1265015.json', '1445816.json', '117584.json', '7230485.json', '8350532.json', '8069870.json', '1858432.json', '851379.json', '10316901.json', '646959.json', '1506269.json', '3157.json', '442457.json', '35960.json', '110017.json', '445941.json', '617385.json', '714761.json', '1806336.json', '655549.json', '2492093.json', '1479781.json', '1634863.json', '798993.json', '1916280.json', '411998.json', '11760283.json', '695557.json', '1355941.json', '120932.json', '9005297.json', '5638282.json', '592044.json', '5405.json', '1633370.json', '1302988.json', '928506.json', '222327.json', '10343.json', '1715566.json', '11754970.json', '11629508.json', '346607.json', '201514.json', '4851579.json', '40265.json', '3093496.json', '4971868.json', '1865764.json', '1998963.json', '10834022.json', '3445809.json', '10356131.json', '4323411.json', '5345007.json', '1197206.json', '29055.json', '1329965.json', '2901532.json', '1760149.json', '2280848.json', '2522491.json', '1091858.json', '134635.json', '1349299.json', '1535818.json', '716358.json', '714312.json', '6623232.json', '1780690.json', '4898231.json', '1371395.json', '1351373.json', '172425.json', '2602402.json', '194033.json', '7469313.json', '4458251.json', '227703.json', '11688860.json', '9564046.json', '9791059.json', '561099.json', '4750663.json', '10831742.json', '11053139.json', '11315649.json', '1370551.json', '407013.json', '4767992.json', '660976.json', '1705053.json', '251900.json', '9812571.json', '219295.json', '5667499.json', '2850583.json', '1074156.json', '18641.json', '307540.json', '2164217.json', '572637.json', '10355992.json', '364890.json', '5370493.json', '510677.json', '3665501.json', '712091.json', '4398071.json', '1893095.json', '1164223.json', '759978.json', '2723285.json', '109150.json', '5564936.json', '9470063.json', '2622060.json', '9173504.json', '914560.json', '656994.json', '818621.json', '547903.json', '2706626.json', '1407738.json', '4947957.json', '9335454.json', '10517295.json', '893479.json', '7881531.json', '1538088.json', '364147.json', '2937899.json', '488767.json', '11197843.json', '3582397.json', '121800.json', '9567942.json', '2732695.json', '10530216.json', '11774387.json', '9499325.json', '742787.json', '471323.json', '11760393.json', '3311710.json', '339223.json', '816520.json', '10265565.json', '1297252.json', '1875175.json', '3306978.json', '3299505.json', '1333217.json', '126536.json', '2783591.json', '1297442.json', '364262.json', '9466.json', '11091246.json', '11462400.json', '1251773.json', '1867133.json', '1383958.json', '9624495.json', '1152665.json', '9691662.json', '928507.json', '4341742.json', '7610848.json', '4130536.json', '11575635.json', '7758791.json', '4749320.json', '440459.json', '3566646.json', '603926.json', '9294481.json', '1314047.json', '1247069.json', '10937360.json', '2553262.json', '466896.json', '9494199.json', '67848.json', '700711.json', '547904.json', '1366801.json', '1972799.json', '59367.json', '5093938.json', '868748.json', '625947.json', '891101.json', '2447102.json', '11591782.json', '9768.json', '10370770.json', '817269.json', '7506917.json', '9564257.json', '1956655.json', '1077855.json', '385968.json', '11039523.json', '1429657.json', '908207.json', '227154.json', '1887751.json', '1030832.json', '1249325.json', '238219.json', '2850878.json', '1968195.json', '542141.json', '1350374.json', '22849.json', '1537622.json', '7984227.json', '4898796.json', '11305593.json', '4272652.json', '7779566.json', '230520.json', '11091257.json', '1402659.json', '1372233.json', '2362139.json', '665178.json', '9569107.json', '82453.json', '8758712.json', '887489.json', '1506240.json', '10066813.json', '9579500.json', '2069649.json', '2344162.json', '6401910.json', '1452883.json', '9547757.json', '11593971.json', '7702950.json', '628355.json', '1170787.json', '2710239.json', '1483562.json', '14516.json', '5719013.json', '1226366.json', '6390179.json', '39004.json', '833281.json', '9476903.json', '8332537.json', '588571.json', '1048435.json', '1862927.json', '1634534.json', '10534896.json', '3924812.json', '748733.json', '5730017.json', '1292113.json', '279870.json', '9816755.json', '3042225.json', '8486342.json', '683685.json', '163340.json', '1940108.json', '1388851.json', '156262.json', '4929657.json', '9373793.json', '39341.json', '401271.json', '1488712.json', '75593.json', '3869727.json', '11644341.json', '6079861.json', '2771300.json', '1401998.json', '1170591.json', '310164.json', '5670551.json', '26558.json', '368969.json', '1556652.json', '2850746.json', '2678776.json', '9765674.json', '4123015.json', '10060620.json', '10804431.json', '8276402.json', '51845.json', '1483191.json', '2120772.json', '9190785.json', '2238747.json', '1215763.json', '23853.json', '1183833.json', '1374796.json', '11091260.json', '2704689.json', '1171135.json', '3726616.json', '944883.json', '2008793.json', '2959054.json', '1065146.json', '1107818.json', '4852974.json', '5783601.json', '10500940.json', '92370.json', '49274.json', '216665.json', '44850.json', '2493219.json', '5716451.json', '5833832.json', '443018.json', '1154216.json', '58074.json', '1194948.json']
# Games
sub_categories['Games'] = ['5382493.json', '2884737.json', '17621.json', '5247604.json', '3947575.json', '5455730.json', '4123738.json', '164865.json', '199445.json', '9696643.json', '2408982.json', '677576.json', '9176105.json', '11165204.json', '7987716.json', '3958012.json', '3152631.json', '1296734.json', '799736.json', '6259438.json', '9817976.json', '1518987.json', '6664085.json', '54279.json', '182663.json', '9759667.json', '8358349.json', '1358785.json', '8588772.json', '5470517.json', '6050996.json', '6015870.json', '2152589.json', '229426.json', '5223705.json', '857048.json', '884985.json', '203472.json', '1307239.json', '397828.json', '1088923.json', '8503253.json', '4356061.json', '5323060.json', '58077.json', '9910240.json', '3707575.json', '2565958.json', '5428575.json', '5841387.json', '464870.json', '7345160.json', '4199636.json', '3196570.json', '6012507.json', '628707.json', '342441.json', '596613.json', '653199.json', '6762497.json', '5484761.json', '743428.json', '11623341.json', '1012218.json', '7804512.json', '843553.json', '1302021.json', '1303260.json', '5050628.json', '38553.json', '7353827.json', '197197.json', '3551983.json', '118764.json', '3887610.json', '11726718.json', '4777666.json', '2015403.json', '51974.json', '11230285.json', '6515.json', '9141651.json', '375899.json', '6365697.json', '3707607.json', '11751263.json', '7759428.json', '9736777.json', '10016343.json', '836853.json', '5631418.json', '761830.json', '1620919.json', '3952215.json', '427091.json', '1560596.json', '3428794.json', '7621350.json', '3604124.json', '5897865.json', '3707610.json', '284645.json', '166183.json', '9862314.json', '6805311.json', '1186717.json', '2778162.json', '4948444.json', '5008169.json', '1602384.json', '987843.json', '229942.json', '389047.json', '5182212.json', '2429397.json', '1663013.json', '11323477.json', '5754641.json', '139805.json', '3725126.json', '948115.json', '4393623.json', '9536601.json', '9724622.json', '3707554.json', '11003981.json', '4430911.json', '3888136.json', '329732.json', '722481.json', '13053.json', '1984.json', '584101.json', '1354011.json', '4368846.json', '6913488.json', '452522.json', '3707574.json', '3517839.json', '7100144.json', '658424.json', '79793.json', '514974.json', '698157.json', '11068368.json', '1354748.json', '329246.json', '132697.json', '1575736.json', '1339002.json', '532780.json', '310202.json', '181449.json', '153296.json', '1949999.json', '1934816.json', '1271660.json', '65765.json', '572761.json', '3437816.json', '10851594.json', '5288241.json', '5798608.json', '11029920.json', '3536962.json', '4927542.json', '77426.json', '216866.json', '3110802.json', '6142426.json', '7557848.json', '10806377.json', '2139394.json', '765444.json', '6813359.json', '4064580.json', '1558422.json', '2611207.json', '2450996.json', '9689276.json', '37015.json', '491128.json', '5490017.json', '126178.json', '20046.json', '11529420.json', '7020930.json', '5895936.json', '1562304.json', '42237.json', '1375462.json', '14841.json', '1901817.json', '1758749.json', '17943.json', '3707553.json', '2139409.json', '464437.json', '4389803.json', '4281081.json', '787333.json', '7668210.json', '411521.json', '4860820.json', '292217.json', '41909.json', '29738.json', '1371899.json', '11733567.json', '11617474.json', '3152457.json', '28607.json', '1545348.json', '7187861.json', '189770.json', '204041.json', '11680967.json', '3162833.json', '5666479.json', '4261579.json', '5331807.json', '46864.json', '612692.json', '6340033.json', '658244.json', '6328594.json', '809707.json', '3424134.json', '4915510.json', '819838.json', '5580581.json', '1322741.json', '4216.json', '158994.json', '3123129.json', '1569476.json', '162305.json', '5129029.json', '11620064.json', '244583.json', '9845.json', '6443291.json', '363592.json', '2600041.json', '3412837.json', '1237482.json', '6539078.json', '10529595.json', '1914571.json', '9769886.json', '3895430.json', '10750902.json', '1352249.json', '671631.json', '3707549.json', '4191758.json', '5643347.json', '4490445.json', '8791463.json', '1622806.json', '4784687.json', '498524.json', '2563469.json', '414580.json', '474728.json', '137353.json', '3002019.json', '6940433.json', '155200.json', '7957023.json', '619197.json', '43410.json', '5460854.json', '71998.json', '495506.json', '42597.json', '2836110.json', '9506382.json', '1100675.json', '374914.json', '108328.json', '1453598.json', '4318325.json', '24429.json', '362481.json', '974227.json', '9701.json', '4282727.json', '28666.json', '927079.json', '304591.json', '3747678.json', '3707573.json', '9731954.json', '542217.json', '1422874.json', '1938433.json', '5402802.json', '6435197.json', '9512051.json', '73320.json', '2601915.json', '11129957.json', '1684418.json', '2992512.json', '4321272.json', '2617087.json', '8608764.json', '1236744.json', '4484939.json', '1277209.json', '6050991.json', '4535.json', '6580413.json', '4859668.json', '6008479.json', '274227.json', '5480617.json', '5344377.json', '5937828.json', '603061.json', '6050987.json', '545879.json', '9829.json', '3408294.json', '6885426.json', '5129032.json', '178713.json', '132500.json', '278551.json', '199842.json', '318161.json', '72419.json', '5950190.json', '9639913.json', '3707548.json', '2997815.json', '6995768.json', '7125236.json', '1574623.json', '5127846.json', '349085.json', '141852.json', '390942.json', '6988377.json', '1374442.json', '403045.json', '385886.json', '5490292.json', '11800520.json', '2424063.json', '30037.json', '2418419.json', '8276563.json', '535262.json', '4887466.json', '424879.json', '6816604.json', '8181623.json', '10361502.json', '7895.json', '9034822.json', '10690148.json', '132808.json', '7902127.json', '3960908.json', '703860.json', '1298561.json', '899216.json', '1254174.json', '45988.json', '8926002.json', '11766910.json', '2559332.json', '7108568.json', '448076.json', '25809.json', '3810179.json', '6979817.json', '227370.json', '9074139.json', '315837.json', '11791111.json', '386422.json', '369583.json', '3442.json', '173256.json', '515882.json', '148239.json', '149428.json', '182469.json', '171441.json', '695597.json', '436215.json', '1283655.json', '211604.json', '11462592.json', '7155529.json', '8317040.json', '659182.json', '1465521.json', '9554703.json', '6296131.json', '6803.json', '9668568.json', '4329972.json', '8520887.json', '6950997.json', '5108371.json', '2327535.json', '31737.json', '9858580.json', '11250879.json', '492234.json', '1236570.json', '4813177.json', '1352553.json', '11767990.json', '453382.json', '9612119.json', '1424151.json', '103839.json', '8547294.json', '193216.json', '1545389.json', '10382303.json', '5014.json', '8091387.json', '187193.json', '8116676.json', '6885429.json', '3772257.json', '9780428.json', '6141110.json', '5551927.json', '529891.json', '10421420.json', '309354.json', '187081.json', '3144846.json', '9547511.json', '200515.json', '492127.json', '347313.json', '9377485.json', '5354092.json', '5879606.json', '1674911.json', '7226155.json', '810230.json', '1204600.json', '6561921.json', '5127776.json', '7014485.json', '699722.json', '1571357.json', '3707547.json', '6203033.json', '841383.json', '1832056.json', '4293114.json', '9640254.json', '3707551.json', '8369164.json', '800334.json', '847715.json', '2931645.json', '9372412.json', '1216806.json', '3115.json', '382970.json', '4308370.json', '3178876.json', '4490538.json', '5288229.json', '418108.json', '3684755.json', '4158065.json', '476644.json', '14834.json', '2152132.json', '6845906.json', '8941937.json', '198285.json', '1595971.json', '6436346.json', '69503.json', '10545458.json', '6714720.json', '5127613.json', '9083.json', '9462862.json', '1999807.json', '988802.json', '64389.json', '98054.json', '4669050.json', '721174.json', '4728829.json', '21582.json', '10575754.json', '9730629.json', '188753.json', '6671493.json', '7378196.json', '5455722.json', '9849935.json', '1325819.json', '640247.json', '8562791.json', '5427333.json', '11189098.json', '6873459.json', '1872078.json', '147205.json', '404042.json', '4308075.json', '4764351.json', '5633456.json', '1146484.json', '29798.json', '1966477.json', '136487.json', '1570767.json', '384329.json', '1400643.json', '1341974.json', '3552109.json', '10389651.json', '1359781.json', '10065074.json', '664099.json', '711701.json', '11182293.json', '77658.json', '2203811.json', '6885388.json', '3543759.json', '5350658.json', '684371.json', '3647489.json', '518482.json', '3557063.json', '1137354.json', '201974.json', '10249810.json', '63748.json', '693322.json', '675276.json', '11755706.json', '11757691.json', '9465324.json', '9341943.json', '9637630.json', '6830997.json', '144890.json', '649677.json', '3331810.json', '6074131.json', '11755141.json', '7597406.json', '6050985.json', '6010698.json', '5481141.json', '576423.json', '6337551.json', '121545.json', '413092.json', '2246830.json', '1647196.json', '389188.json', '664148.json', '3829172.json', '3126689.json', '180708.json', '4763456.json', '784707.json', '1302878.json', '1643182.json', '19211.json', '618681.json', '8825041.json', '8345753.json', '2064206.json', '11790190.json', '4513443.json', '197517.json', '4280781.json', '6860940.json', '2619668.json', '3231181.json', '1831102.json', '3707550.json', '67438.json', '3678915.json', '9421758.json', '3382093.json', '6130642.json', '11111635.json', '449075.json', '5951222.json', '26071.json', '1675593.json', '6074111.json', '845768.json', '574828.json', '279821.json', '8543844.json', '3917499.json', '5455538.json', '5482389.json', '1326255.json', '15013.json', '1188920.json', '535671.json', '11733773.json', '9913998.json', '5249783.json', '7742762.json', '61581.json', '10826032.json', '5638391.json', '8954112.json', '357514.json', '263796.json', '2226801.json', '4044515.json', '357113.json', '8348244.json', '10094368.json', '1540431.json', '178567.json', '7374706.json', '967392.json', '417423.json', '136388.json', '1121170.json', '10865104.json', '6673996.json', '3707577.json', '4075613.json', '104972.json', '47416.json', '699968.json', '1470643.json', '6336994.json', '11585997.json', '664765.json', '41215.json', '229598.json', '4300091.json', '689991.json', '4238267.json', '1657282.json', '245380.json', '640646.json', '5592648.json', '9316489.json', '11345753.json', '5323776.json', '314281.json', '352265.json', '8901641.json', '2173038.json', '411883.json', '10568988.json', '270060.json', '44580.json', '4232980.json', '276272.json', '349346.json', '4985570.json', '626039.json', '204376.json', '2494322.json', '6367722.json', '10429831.json', '10861567.json', '5484889.json', '190580.json', '7787813.json', '1831892.json', '448177.json', '9703895.json', '1680393.json', '1589688.json', '47908.json', '11282023.json', '1113913.json', '320316.json', '7946544.json', '10311078.json', '1857541.json', '1352353.json', '6854624.json', '9667926.json', '696193.json', '8776904.json', '9853489.json', '882189.json', '380320.json', '381474.json', '11208283.json', '546159.json', '1260414.json', '3714402.json', '516317.json', '119399.json', '10893493.json', '9872792.json', '5634663.json', '5219637.json', '461722.json', '8728305.json', '1341704.json', '1216694.json', '5795921.json', '3912473.json', '572515.json', '11399561.json', '5399454.json', '1376135.json', '1212100.json', '824504.json', '4251559.json', '944134.json', '8964465.json', '8642953.json', '6218551.json', '8814393.json', '243961.json', '5463477.json', '335302.json', '3506898.json', '61682.json', '181083.json', '2772845.json', '763863.json', '6178201.json', '170844.json', '78122.json', '1331892.json', '9715147.json', '6443426.json', '121631.json', '1361388.json', '42434.json', '270903.json', '6225468.json', '397309.json', '6043073.json', '151421.json', '102556.json', '1095719.json', '2256210.json', '1108889.json', '824569.json', '101100.json', '5841594.json', '5906687.json', '3164223.json', '3707604.json', '2878308.json', '8520880.json', '3151839.json', '1939461.json', '1194030.json', '6830968.json', '8386830.json', '1151595.json', '709337.json', '509903.json', '270061.json', '1439139.json', '11780708.json', '11129986.json', '29731.json', '1546613.json', '11557774.json', '3784723.json', '5996860.json', '932301.json', '395894.json', '1502019.json', '9230637.json', '1555341.json', '1508697.json', '833760.json', '333665.json', '6939779.json', '10095359.json', '5288238.json', '3594341.json', '1353021.json', '227616.json', '241356.json', '46990.json', '129431.json', '2217356.json', '178740.json', '3552135.json', '1020624.json', '11801860.json', '8940008.json', '11259645.json', '2039575.json', '405290.json', '4382102.json', '4158520.json', '11752339.json', '3832041.json', '4667321.json', '478870.json', '588415.json', '801806.json', '702804.json', '20195.json', '695049.json', '413606.json', '3707576.json', '6845540.json', '3205801.json', '806443.json', '2402765.json']
# Youth Dating
sub_categories['Youth Dating'] = ['279994.json', '3491473.json', '115926.json', '968042.json', '360750.json', '205861.json', '115434.json', '1470047.json', '2891523.json', '2498196.json', '3525119.json', '1852736.json', '2934025.json', '266972.json', '32543.json', '345035.json', '1787385.json', '47382.json', '397019.json', '573079.json', '1933608.json', '13013.json', '91178.json', '451913.json', '1296041.json', '727552.json', '70706.json', '684285.json', '610538.json', '284306.json', '1845769.json', '1733954.json', '57770.json', '1703635.json', '1388487.json', '2220883.json', '932226.json', '313659.json', '1898202.json', '4732779.json', '2413710.json', '1584554.json', '1198749.json', '139078.json', '4586.json', '124831.json', '159140.json', '794563.json', '131388.json', '1058698.json', '23830.json', '405637.json', '687484.json', '96628.json', '5235367.json', '156098.json', '630475.json', '754840.json', '407333.json', '768684.json', '18259.json', '285097.json', '95028.json', '13012.json', '791999.json', '155698.json', '1301323.json', '737284.json', '574752.json', '1643260.json', '196181.json', '2403947.json', '188384.json', '1082705.json', '125810.json', '1054875.json', '861069.json', '34584.json', '3631118.json', '3861421.json', '665527.json', '51216.json', '1087454.json', '71474.json', '810645.json', '687826.json', '878032.json', '2221661.json', '1655603.json', '15934.json', '76519.json', '37453.json', '305493.json', '1124317.json', '32981.json', '509782.json', '1592399.json', '228488.json', '1153904.json', '5978628.json', '86999.json', '1778032.json', '671765.json', '5385300.json', '11327357.json', '668285.json', '257202.json', '136482.json', '1500607.json', '121570.json', '97004.json', '690055.json', '1512143.json', '101715.json', '694807.json', '1844294.json', '1083349.json', '14787.json', '1934535.json', '5057893.json', '159690.json', '1865789.json', '1792066.json', '1088712.json', '716730.json', '2988329.json', '211445.json', '255998.json', '3701385.json', '260290.json', '597993.json', '661484.json', '1033040.json', '441169.json', '642022.json', '10357503.json', '5824059.json', '1198104.json', '2817773.json', '1242039.json', '492596.json', '7331401.json', '637285.json', '393695.json', '1910089.json', '34278.json', '147753.json', '251996.json', '30186.json', '460784.json', '8186450.json', '125801.json', '776039.json', '1217736.json', '1273027.json', '559322.json', '379672.json', '2991850.json', '2126227.json', '604970.json', '62835.json', '18843.json', '1534115.json', '819835.json', '226238.json', '1708971.json', '1746808.json', '689214.json', '953143.json', '467257.json', '677926.json', '320221.json', '2419547.json', '866006.json', '1639420.json', '12189.json', '1055151.json', '2255496.json', '653798.json', '1763064.json', '742603.json', '280495.json', '717136.json', '82952.json', '1573484.json', '235361.json', '47327.json', '61184.json', '189835.json', '22335.json', '477670.json', '1185453.json', '347255.json', '2025708.json', '2084800.json', '4000777.json', '524330.json', '347982.json', '93904.json', '769752.json', '4638210.json', '58396.json', '1682016.json', '3792.json', '1181001.json', '1972250.json', '1616011.json', '723279.json', '3395.json', '831256.json', '112211.json', '2071006.json', '74686.json', '1398587.json', '260553.json', '172700.json', '356484.json', '1712309.json', '180590.json', '1184001.json', '1821494.json', '331444.json', '70329.json', '1271393.json', '528916.json', '485174.json', '49746.json', '4401025.json', '849875.json', '1907703.json', '2391253.json', '493720.json', '1327306.json', '1624925.json', '114450.json', '4064004.json', '1497792.json', '1871793.json', '1096702.json', '3104700.json', '17536.json', '46610.json', '783743.json', '37616.json', '1741272.json', '1309632.json', '129820.json', '675349.json', '6819.json', '269656.json', '131641.json', '96932.json', '1311691.json', '51638.json', '308836.json', '719263.json', '5061917.json', '2184405.json', '1120890.json', '11030.json', '1336665.json', '128071.json', '2660480.json', '392614.json', '1343509.json', '373986.json', '1090746.json', '151943.json', '993234.json', '298438.json', '16936.json', '397010.json', '605709.json', '1810429.json', '570133.json', '793851.json', '27007.json', '1540305.json', '277720.json', '1060529.json', '1150204.json', '382793.json', '577766.json', '88043.json', '768527.json', '406394.json', '324915.json', '172812.json', '645009.json', '243475.json', '1266601.json', '1099478.json', '257524.json', '1824880.json', '1378212.json', '677882.json', '526134.json', '146458.json', '32065.json', '2324267.json', '735643.json', '79488.json', '92839.json', '371037.json', '2875896.json', '1350892.json', '9523.json', '453546.json', '925289.json', '443546.json', '1988759.json', '695444.json', '452879.json', '190246.json', '37436.json', '3888.json', '1194676.json', '2165569.json', '275425.json', '520252.json', '1684775.json', '5716446.json', '3669661.json', '399176.json', '1252857.json', '1488066.json', '452167.json', '107132.json', '204462.json', '1899775.json', '4736535.json', '42012.json', '2611300.json', '168956.json', '1333932.json', '82873.json', '1680167.json', '635245.json', '359468.json', '472046.json', '2945083.json', '224164.json', '597305.json', '960145.json', '590993.json', '22633.json', '38843.json', '12397.json', '191738.json', '1176517.json', '6311829.json', '5050295.json', '2433865.json', '199099.json', '1545622.json', '6392294.json', '2964249.json', '1142967.json', '150837.json', '1586228.json', '2120840.json', '7543687.json', '255621.json', '37671.json', '16439.json', '1956423.json', '13457.json', '49214.json', '724782.json', '102582.json', '723786.json', '784665.json', '3835527.json', '1180643.json', '94154.json', '63241.json', '506896.json', '137819.json', '2310769.json', '58500.json', '659939.json', '47533.json', '452238.json', '70457.json', '457387.json', '95401.json', '1683269.json', '655936.json', '2061891.json', '2169013.json', '784424.json', '1735600.json', '324591.json', '1259611.json', '2245187.json', '510299.json', '415195.json', '325436.json', '7233825.json', '1951168.json', '1623880.json', '99947.json', '109814.json', '1156657.json', '220055.json', '4413785.json', '827723.json', '1445473.json', '281170.json', '1427259.json', '765885.json', '646212.json', '306946.json', '1761552.json', '94174.json', '4145892.json', '44846.json', '845975.json', '677735.json', '2549286.json', '134995.json', '1249534.json']
# Youth Culture
sub_categories['Youth Culture'] = ['4365900.json', '399908.json', '2191091.json', '138999.json', '3330678.json', '55110.json', '895603.json', '971760.json', '189146.json', '431298.json', '188147.json', '210507.json', '2495213.json', '424055.json', '111355.json', '704967.json', '58094.json', '53322.json', '7256269.json', '2028297.json', '743452.json', '53362.json', '665878.json', '1842691.json', '298975.json', '224018.json', '43533.json', '4362909.json', '329698.json', '145909.json', '5617.json', '613728.json', '283572.json', '8094071.json', '257003.json', '2221026.json', '11832.json', '203625.json', '826219.json', '7237.json', '943743.json', '235097.json', '60434.json', '46448.json', '40123.json', '136958.json', '83991.json', '171354.json', '855205.json', '370078.json', '77899.json', '215133.json', '2585478.json', '1541079.json', '920010.json', '705491.json', '324058.json', '128177.json', '531168.json', '105936.json', '94999.json', '258349.json', '78613.json', '1615759.json', '698916.json', '1727233.json', '6676.json', '1792641.json', '478177.json', '377786.json', '149737.json', '618482.json', '1518040.json', '850925.json', '1257739.json', '209687.json', '443010.json', '225770.json', '434341.json', '408015.json', '963146.json', '174317.json', '49250.json', '11509613.json', '288360.json', '46489.json', '594997.json', '652466.json', '121266.json', '433956.json', '514394.json', '9203078.json', '1751726.json', '416638.json', '14863.json', '1794153.json', '10725.json', '637116.json', '199046.json', '1085905.json', '663423.json', '3231.json', '470617.json', '142010.json', '433408.json', '480245.json', '1751418.json', '1237950.json', '1484146.json', '8958058.json', '191569.json', '28307.json', '3817.json', '112839.json', '1888047.json', '517501.json', '223720.json', '509227.json', '516214.json', '2628676.json', '29652.json', '98777.json', '169767.json', '1603401.json', '35400.json', '355719.json', '337975.json', '5543394.json', '904176.json', '88750.json', '151504.json', '5967566.json', '91971.json', '1612237.json', '166159.json', '47178.json', '132909.json', '398116.json', '250272.json', '896723.json', '216384.json', '70682.json', '204069.json', '277117.json', '79620.json', '384123.json', '382022.json', '1174681.json', '25477.json', '1760119.json', '1065473.json', '218088.json', '781519.json', '176302.json', '1532476.json', '2244846.json', '269149.json', '1886546.json', '127550.json', '51783.json', '2199549.json', '3069600.json', '45715.json', '465887.json', '403997.json', '253296.json', '74320.json', '1222241.json', '662864.json', '121447.json']
# Social Interactions for Youth
sub_categories['Social Interactions for Youth'] = ['122993.json', '1658533.json', '1818661.json', '1058791.json', '5269412.json', '530179.json', '288016.json', '85361.json', '841295.json', '85316.json', '1918221.json', '6427877.json', '448026.json', '188632.json', '129756.json', '5133136.json', '195369.json', '4263105.json', '1211485.json', '1340212.json', '1193041.json', '932661.json', '680382.json', '5202468.json', '653474.json', '83017.json', '2956403.json', '532427.json', '633756.json', '6391570.json', '1208702.json', '311385.json', '1156356.json', '1368430.json', '81925.json', '959876.json', '8828191.json', '838534.json', '425642.json', '76904.json', '6521995.json', '5533972.json', '516850.json', '47388.json', '82011.json', '5638832.json', '1670771.json', '1567777.json', '513367.json', '3859530.json', '6197686.json', '272864.json', '1003383.json', '37593.json', '2165757.json', '291772.json', '5934091.json', '842332.json', '130053.json', '170933.json', '843535.json', '27033.json', '623434.json', '1950823.json', '319245.json', '515857.json', '16187.json', '123761.json', '277186.json', '23271.json', '8152950.json', '448277.json', '1541260.json', '2880896.json', '47959.json', '46519.json', '101756.json', '1862074.json', '1162704.json', '675514.json', '628689.json', '1658665.json', '18114.json', '657445.json', '2077667.json', '161835.json', '32785.json', '27354.json', '8569098.json', '5449756.json', '125655.json', '66377.json', '34407.json', '10227561.json', '670598.json', '897379.json', '2599225.json', '443531.json', '641457.json', '11675482.json', '209105.json', '1191737.json', '421788.json', '1375115.json', '227186.json', '7024.json', '1338894.json', '4712112.json', '300882.json', '504815.json', '625537.json', '1163840.json', '46959.json', '690257.json', '630133.json', '2530751.json', '2776093.json', '826552.json', '1204007.json', '8495010.json', '60972.json', '251678.json', '1943768.json', '159860.json', '831176.json', '312764.json', '102795.json', '2265434.json', '308406.json', '8471001.json', '1221374.json', '1213205.json', '1267294.json', '6561527.json', '94339.json', '2182718.json', '143975.json', '263354.json', '7728339.json', '228296.json', '1164839.json', '1519751.json', '722421.json', '20951.json', '166908.json', '8484867.json', '172176.json', '1206358.json']
# Friends
sub_categories['Friends'] = ['1124245.json', '1162425.json', '9346329.json', '610559.json', '2149214.json', '9516243.json', '2320780.json', '481187.json', '4733.json', '111135.json', '758403.json', '15221.json', '164367.json', '2115690.json', '8276987.json', '460151.json', '4360954.json', '634753.json', '11223.json', '5719748.json', '8415118.json', '2094759.json', '7963.json', '1186999.json', '15885.json', '770866.json', '4910387.json', '38494.json', '5900269.json', '3989278.json', '2257195.json', '30636.json', '203385.json', '5536832.json', '4530330.json', '50830.json', '946576.json', '17093.json', '2978090.json', '8310583.json', '270668.json', '148204.json', '1735479.json', '3990126.json', '1987796.json', '304020.json', '435280.json', '7566949.json', '8803204.json', '28841.json', '1777165.json', '310167.json', '317973.json', '395459.json', '1330393.json', '82811.json', '208225.json', '1069184.json', '1120909.json', '2151756.json', '195611.json', '576391.json', '1311948.json', '1860236.json', '257035.json', '1181512.json', '1686680.json', '215704.json', '136932.json', '48451.json', '3631753.json', '11088370.json', '45351.json', '665423.json', '6356729.json', '1477613.json', '314241.json', '281762.json', '526452.json', '386541.json', '1925211.json', '965077.json', '395746.json', '583261.json', '585835.json', '1518945.json', '5417554.json', '1860543.json', '113762.json', '4554178.json', '1651482.json', '646338.json', '2522162.json', '5821964.json', '519956.json', '273927.json', '1552474.json', '163646.json', '8350030.json', '112617.json', '1376649.json', '805026.json', '8777700.json', '257474.json', '2200320.json', '38607.json', '685492.json', '228591.json', '520098.json', '523964.json', '12151.json', '374631.json', '1806073.json', '819357.json', '598109.json', '380349.json', '2208546.json', '1286883.json', '1170255.json', '5477519.json', '5767235.json', '12976.json', '5919941.json', '2581587.json', '431932.json', '2008217.json', '781615.json', '5744922.json', '411203.json', '847218.json', '305752.json', '1718618.json', '3032769.json', '310602.json', '39962.json', '211058.json', '1232985.json', '2067673.json', '7544561.json', '936853.json', '1837237.json', '2213890.json', '42578.json', '8374365.json', '2235326.json', '864903.json', '1593736.json', '309208.json', '1800041.json', '1764484.json', '1391087.json', '481544.json', '8634563.json', '18506.json', '5923884.json', '459145.json', '2702722.json', '8520467.json', '1331033.json', '6277701.json', '8805549.json', '740464.json', '5599847.json', '1698397.json', '8532860.json', '2488650.json', '560131.json', '655695.json', '92749.json', '1385816.json', '45951.json', '344434.json', '118610.json', '10443287.json', '2558866.json', '3377784.json', '1099006.json', '75240.json', '8827619.json', '8873414.json', '60392.json', '8005507.json', '2131393.json', '3892045.json', '5500356.json', '1873575.json', '1109361.json', '32335.json', '35608.json', '207729.json', '8635186.json', '1240818.json', '1856617.json', '7401455.json', '443529.json', '503813.json', '3738991.json', '1025678.json', '438460.json', '41226.json', '62950.json', '761588.json', '8806055.json', '6288251.json', '893136.json', '10342735.json', '146635.json', '1208224.json', '9144085.json', '15985.json', '1426044.json', '3268679.json', '8297670.json', '135566.json', '933389.json', '8310155.json', '484266.json', '1775062.json', '1447561.json', '22269.json', '82361.json', '8425800.json', '376252.json', '134253.json', '155039.json', '2203607.json', '2122271.json', '288638.json', '1602876.json', '8041.json', '3528884.json', '1136143.json', '4506123.json', '1060787.json', '110848.json', '354837.json', '2288023.json', '1559148.json', '570124.json', '377642.json', '20852.json', '4429955.json', '4001190.json', '124286.json', '3293113.json', '718432.json', '1779109.json', '1393997.json', '1179846.json', '132191.json', '97699.json', '1124344.json', '243204.json', '262918.json', '7692411.json', '8736110.json', '48208.json', '256753.json', '1865505.json', '322009.json', '172520.json', '96698.json', '5391957.json', '39062.json', '774512.json', '311694.json', '545986.json', '132543.json', '652646.json', '149261.json', '8185814.json', '1599046.json', '270698.json', '3741039.json', '1394680.json', '1366534.json', '5479590.json', '166294.json', '325892.json', '579888.json', '31171.json', '2306357.json', '555271.json', '838608.json', '1275596.json', '626531.json', '66141.json', '1961716.json', '932833.json', '20218.json', '1510901.json', '584129.json', '2229977.json', '437884.json', '159778.json', '214582.json', '148755.json', '4692634.json', '5618452.json', '544847.json', '5329253.json', '5116109.json', '1289396.json', '401155.json', '44386.json', '425534.json', '102127.json', '8508132.json', '6495274.json', '936210.json', '253995.json', '1565021.json', '153186.json', '485168.json', '4480115.json', '23813.json', '5339176.json', '31361.json', '147448.json', '5667041.json', '8237181.json', '566347.json', '3617571.json', '2720318.json', '1593840.json', '165313.json', '7234379.json', '8315691.json', '3617526.json', '400949.json', '57027.json', '269719.json', '531137.json', '5923376.json', '1200429.json', '1747504.json', '515583.json', '1261897.json', '704733.json', '9682064.json', '567982.json', '1029433.json', '1005225.json', '4907.json', '8455119.json', '1388902.json', '6614166.json', '27365.json', '1278530.json', '7897247.json', '117810.json', '577428.json', '8371904.json', '8316585.json', '7957704.json', '449233.json', '1657098.json', '2716505.json', '1509337.json', '671090.json', '2717986.json', '11103049.json', '163415.json', '1287940.json', '54763.json', '472067.json', '1385166.json', '1135359.json', '2236503.json', '344467.json', '1702504.json', '1329201.json', '97670.json', '447124.json', '131304.json', '1326674.json', '86870.json', '546379.json', '156147.json', '3549169.json', '793713.json', '668602.json', '267233.json', '162834.json', '166877.json', '6220670.json', '4765183.json', '375087.json', '1428084.json']
# Youth and Family
sub_categories['Youth and Family'] = ['138472.json', '1135950.json', '87235.json', '1029754.json', '99114.json', '2147359.json', '4817254.json', '2282125.json', '1037896.json', '649941.json', '170041.json', '1715758.json', '428098.json', '1305233.json', '166814.json', '492925.json', '3223696.json', '8387393.json', '6659754.json', '708676.json', '1405890.json', '842162.json', '98155.json', '20569.json', '1521147.json', '205499.json', '1209538.json', '676506.json', '31719.json', '1341717.json', '445794.json', '93728.json', '6060192.json', '172981.json', '665605.json', '2234350.json', '1381435.json', '281544.json', '370214.json', '1034930.json', '268638.json', '150487.json', '2157758.json', '41397.json', '369166.json', '296809.json', '35916.json', '1728650.json', '188157.json', '466082.json', '39349.json', '516099.json', '344475.json', '64963.json', '27382.json', '11205729.json', '2206083.json', '2075881.json', '159890.json', '5599735.json', '9767753.json', '5189174.json', '5247620.json', '162995.json', '6046630.json', '5862123.json', '652348.json', '18227.json', '6285588.json', '98920.json', '485748.json', '84112.json', '2300778.json', '5092358.json', '739196.json', '6225.json', '155756.json', '8392321.json', '5372246.json', '551454.json', '1153501.json', '849707.json', '4445458.json', '5770300.json', '51117.json', '1857248.json', '11389785.json', '132943.json', '26155.json', '5541592.json', '4653926.json', '1068355.json', '1301673.json', '3529.json', '596155.json', '841245.json', '115796.json', '1175280.json', '6959826.json', '748895.json', '1382987.json', '2511181.json', '1803994.json', '88009.json', '2028390.json', '60952.json', '68461.json', '28382.json', '724669.json', '6225813.json', '87943.json', '1358837.json', '782067.json', '1298288.json', '4750404.json', '11339279.json', '2726849.json', '1552723.json', '209007.json', '9899283.json', '42802.json', '34713.json', '575065.json', '456037.json', '2535723.json', '22669.json']
# Social Events for Youth
sub_categories['Social Events for Youth'] = ['866105.json', '40172.json', '528545.json', '264685.json', '2998378.json', '7391305.json', '431283.json', '1538612.json', '163769.json', '231719.json', '4229506.json', '268849.json', '893298.json', '789808.json', '1597965.json', '790957.json', '5565374.json', '288596.json', '1686629.json', '1424240.json', '3862953.json', '5722351.json', '687342.json', '518730.json', '964381.json', '2067281.json', '743199.json', '416616.json', '4954641.json', '1837923.json', '547280.json', '1942147.json', '2595815.json', '1365454.json', '1344889.json', '2166593.json', '733062.json', '1338793.json', '155915.json', '2680911.json', '1467588.json', '1131963.json', '1589635.json', '6642714.json', '147524.json', '2090456.json', '8381311.json', '245148.json', '104329.json', '2376280.json', '478441.json', '2147330.json', '1366145.json', '677266.json', '2286504.json', '35607.json', '164066.json', '1197708.json', '1300890.json', '2495719.json', '4572218.json', '7735716.json', '8037182.json', '372969.json', '942195.json', '1183395.json', '97569.json', '3328282.json', '1767690.json', '117372.json', '7367.json', '1567677.json', '2775698.json', '2172920.json', '183206.json', '611487.json', '667939.json', '112645.json', '330911.json', '155622.json', '1399893.json', '21008.json', '433642.json', '1553759.json', '1063876.json', '303898.json', '1121551.json', '1583375.json', '3031843.json', '158437.json', '4384869.json']
# Looking Good
sub_categories['Looking Good'] = ['4811617.json', '1674722.json', '687333.json', '37757.json', '809392.json', '1863323.json', '2491745.json', '423753.json', '791776.json', '2819435.json', '1958901.json', '47336.json', '744800.json', '1341005.json', '6013946.json', '1597050.json', '431811.json', '450114.json', '8255907.json', '19018.json', '11738290.json', '1476649.json', '1089352.json', '1152397.json', '1059713.json', '51328.json', '6076.json', '285655.json', '67342.json', '4273657.json', '599237.json', '159628.json', '2487984.json', '973825.json', '22586.json', '424622.json', '71388.json', '443565.json', '792500.json', '1662721.json', '2821787.json', '816328.json', '19680.json', '367509.json', '67284.json', '8455672.json', '40859.json', '296954.json', '8073802.json', '740502.json', '704534.json', '1180279.json', '696044.json', '843486.json', '5267351.json', '42632.json', '111890.json', '521941.json', '867045.json', '815053.json', '1804386.json', '93589.json', '8855641.json', '1083351.json', '9523518.json', '1228959.json', '1660637.json', '638489.json', '974316.json', '303308.json', '5311732.json', '1111641.json', '1659160.json', '446523.json', '483086.json', '72522.json', '47683.json', '532486.json', '2900850.json', '1149370.json', '645669.json', '417609.json', '695052.json', '11311801.json', '710418.json', '720283.json', '721610.json', '11767807.json', '1535366.json', '1270301.json', '184468.json', '725825.json', '853927.json', '57342.json', '1629330.json', '374045.json']
# Social Stuff
sub_categories['School Stuff'] = ['890285.json', '3316822.json', '192527.json', '29374.json', '79706.json', '533104.json', '1545483.json', '867217.json', '41407.json', '1527354.json', '2182994.json', '3308200.json', '672735.json', '651055.json', '9757911.json', '8780521.json', '527242.json', '50847.json', '1310102.json', '159634.json', '449800.json', '392515.json', '803357.json', '622998.json', '2150202.json', '849922.json', '60194.json', '1228907.json', '2690849.json', '743644.json', '2106589.json', '586347.json', '1278122.json', '388552.json', '1560217.json', '336852.json', '74856.json', '306508.json', '1174744.json', '591292.json', '43119.json', '1140962.json', '161578.json', '1597143.json', '151199.json', '1081719.json', '8662.json', '282441.json', '7826423.json', '2092907.json', '147518.json', '333978.json', '238693.json', '296855.json', '4783077.json', '70510.json', '425405.json', '8783005.json', '5252149.json', '996086.json', '41277.json', '75728.json', '84156.json', '4242204.json', '4790354.json', '1939789.json', '125847.json', '41187.json', '9542274.json', '641957.json', '8603.json', '1299585.json', '122316.json', '2176813.json', '83324.json', '1920750.json', '630101.json', '5918087.json', '2334015.json', '619145.json', '3885860.json', '3272108.json', '6458469.json', '840206.json', '2133268.json', '1034195.json', '552671.json', '206928.json', '99667.json', '1569362.json', '7603359.json', '1325513.json', '631998.json', '67929.json', '1593393.json', '91294.json', '45065.json', '525672.json', '1603802.json', '835392.json', '554232.json', '2647856.json', '430138.json', '1500152.json', '1501578.json', '128680.json', '504617.json', '148799.json', '188252.json', '52796.json', '1020973.json', '8583.json', '1058754.json', '536496.json', '347960.json', '1927483.json', '8831596.json', '318863.json', '3661916.json', '444721.json', '596859.json', '1275673.json', '196630.json', '5121100.json', '10912354.json', '4924353.json', '1253583.json', '1241100.json', '25813.json', '811947.json', '41986.json', '337927.json', '2558997.json', '4323648.json', '1023276.json', '2544672.json', '475883.json', '3894575.json', '723977.json', '740521.json', '951781.json', '195420.json', '95649.json', '1558023.json', '36655.json', '8923373.json', '1686632.json', '30040.json', '414016.json', '154666.json', '698736.json', '766188.json', '140019.json', '5603997.json', '72706.json', '128639.json', '4270447.json', '791264.json', '8842426.json', '5475134.json', '1365635.json', '1269870.json', '511394.json', '7453108.json', '481708.json', '1242608.json', '457340.json', '5598962.json', '909366.json', '889781.json', '5474123.json', '721614.json', '1202000.json', '3125010.json', '9308.json', '886375.json', '427169.json', '939897.json', '5256498.json', '957226.json', '2561289.json', '47308.json', '156727.json', '1310521.json', '45698.json', '6016055.json', '4283397.json', '351394.json', '4354306.json', '635536.json', '936867.json', '7013.json', '1360818.json', '7156.json', '1277174.json', '528633.json', '856348.json', '734065.json', '705387.json', '1431719.json', '167518.json', '1932705.json', '156245.json', '211959.json', '555895.json', '89583.json', '164461.json', '290766.json', '41395.json', '877356.json', '5058115.json', '2182752.json', '5166775.json', '9783048.json', '375106.json', '4721.json', '2226524.json', '706704.json', '245543.json', '9670968.json', '670824.json', '551697.json', '7395226.json', '8560241.json', '1565876.json', '3254074.json', '2336313.json', '285709.json', '2185393.json', '84975.json', '3031957.json', '9487248.json', '8073376.json', '1570875.json', '654792.json', '1384053.json', '5078824.json', '609634.json', '3768266.json', '50931.json', '1219025.json', '1537023.json', '636136.json', '35718.json', '1408906.json', '363564.json', '721179.json', '414777.json', '1219658.json', '1541922.json', '394962.json', '1665803.json', '30920.json', '1491431.json', '1875118.json', '1379205.json', '7470484.json', '217879.json', '835283.json', '1185650.json', '721817.json', '7552802.json', '1274169.json', '6584228.json', '3823485.json', '90073.json', '114876.json', '142345.json', '378358.json', '306228.json', '2755541.json', '969186.json', '4944038.json', '167135.json', '1365581.json', '5790446.json', '2369714.json', '648630.json', '596797.json', '610740.json', '22776.json', '1613902.json', '3115224.json', '732011.json', '1217047.json', '65479.json', '5161757.json', '49629.json', '760448.json', '563692.json', '8709064.json', '428166.json', '15741.json', '1674297.json', '1075219.json', '3693809.json', '3273249.json', '657422.json', '1609436.json', '5428536.json', '4947659.json', '79708.json', '9267.json', '152723.json', '1221509.json', '66015.json', '753312.json', '5416976.json', '2563533.json', '1186282.json', '112757.json', '536489.json', '375048.json', '204005.json', '440481.json', '4754191.json', '1580312.json', '7532287.json', '346440.json', '65710.json', '521233.json', '10186.json', '521321.json', '2721764.json', '647264.json', '180352.json', '201505.json', '1698937.json', '46066.json', '1001134.json', '398547.json', '1155667.json', '2312349.json', '132569.json', '1540694.json', '191308.json', '38136.json', '204770.json', '7515906.json', '3710909.json', '1812384.json', '527282.json', '4149976.json', '444837.json', '633105.json', '764580.json', '657478.json', '441176.json', '1174823.json', '111321.json', '334837.json', '225914.json', '1658789.json', '1593288.json', '617460.json', '5154547.json', '5013592.json', '1664865.json', '46482.json', '6078944.json', '199714.json', '1945691.json', '3450417.json', '206331.json', '1414942.json']
# Fandom
sub_categories['Fandom'] = ['273475.json', '5461146.json', '9991439.json', '670876.json', '4596749.json', '1770550.json', '9740082.json', '4198251.json', '392503.json', '97472.json', '652028.json', '2229724.json', '2190819.json', '6415855.json', '614957.json', '525623.json', '1211646.json', '131370.json', '4074671.json', '6216465.json', '5981145.json', '8255244.json', '223964.json', '749970.json', '9585467.json', '13511.json', '589226.json', '1826122.json', '1116319.json', '4409349.json', '1826118.json', '1182324.json', '391181.json', '1640790.json', '124283.json', '11593.json', '8337766.json', '2609811.json', '2959067.json', '1085360.json', '11527214.json', '2522313.json', '174269.json', '5833180.json', '8060871.json', '5480817.json', '931482.json', '1608917.json', '143710.json', '1987374.json', '2172093.json', '927184.json', '114999.json', '6030283.json', '2334655.json', '2185943.json', '2221919.json', '3898151.json', '1667308.json', '995272.json', '5881368.json', '1922614.json', '6903601.json', '139321.json', '5945577.json', '4359761.json', '671240.json', '24127.json', '309793.json', '535258.json', '2439807.json', '2483456.json', '4653049.json', '1777136.json', '419996.json', '468284.json', '1960385.json', '4596700.json', '3760753.json', '6690561.json', '4596716.json', '11323852.json', '7384699.json', '1082096.json', '620597.json', '9897035.json', '2219621.json', '2410909.json', '6404826.json', '6098147.json', '5296922.json', '3873249.json', '6368517.json', '5183935.json', '234303.json', '8806453.json', '11527411.json', '9984167.json', '45709.json', '6860176.json', '6098479.json', '5658962.json', '5770444.json', '6427603.json', '6095288.json', '901837.json', '112916.json', '241567.json', '2547702.json', '230263.json', '1421695.json', '4951.json', '608425.json', '206081.json', '6787712.json', '6301541.json', '11527495.json', '6911435.json', '9727431.json', '27867.json', '428761.json', '381048.json', '6917518.json', '299556.json', '3054353.json', '10021510.json', '33384.json', '1307739.json', '10141675.json', '5763906.json', '6142449.json', '7438161.json', '8650519.json', '1169303.json', '8591679.json', '1719221.json', '71525.json']
# Parties
sub_categories['Parties'] = ['4712615.json', '898965.json', '1911919.json', '656114.json', '412184.json', '207667.json', '44255.json', '2583457.json', '250180.json', '21093.json', '47117.json', '1308836.json', '2033213.json', '2179765.json', '1688579.json', '9691515.json', '1938988.json', '2089059.json', '143130.json', '1041210.json', '169963.json', '1067363.json', '5958495.json', '5768304.json', '390973.json', '303970.json', '1285138.json', '1289361.json', '1167484.json', '571253.json', '948968.json', '442606.json', '1281839.json', '1829326.json', '8296361.json', '26366.json', '1143973.json', '10244928.json', '280893.json', '7885.json', '1312866.json', '443329.json', '4405464.json', '1819207.json', '4329449.json', '418750.json', '26833.json', '56398.json', '434111.json', '11241353.json', '235609.json', '8731188.json', '96096.json', '3506567.json', '8505037.json', '902828.json', '8211136.json', '470074.json', '5811783.json', '1943985.json', '622055.json', '1332988.json', '1977577.json', '1219851.json', '663332.json', '61531.json', '118600.json', '11704561.json', '2289660.json', '404006.json', '3394374.json', '8325316.json', '1629826.json', '396351.json', '9782975.json', '6844001.json', '2836204.json', '501205.json', '8211189.json', '878279.json', '501213.json', '641458.json', '24809.json', '227371.json', '1078511.json', '592545.json', '61490.json', '1534948.json', '8782812.json', '1392393.json', '2863920.json', '1038505.json', '11767766.json', '307041.json', '78107.json', '6417874.json', '3859.json', '631613.json', '5225340.json', '297520.json', '214831.json', '64519.json', '544510.json', '9123925.json', '7965.json', '2541061.json', '2308107.json', '3991454.json', '20200.json', '901058.json', '2596054.json', '5317703.json', '9694478.json', '7026.json', '1758396.json', '2514650.json', '470068.json', '1133529.json', '1319349.json', '8798317.json', '301643.json', '26419.json', '8099126.json', '1908436.json', '4911381.json', '1687.json', '2615761.json', '100754.json', '1596387.json', '3048817.json', '10977449.json', '130622.json', '1518774.json', '8293372.json', '209058.json', '2343336.json', '158979.json', '10415305.json', '519836.json', '2175308.json', '348863.json', '1473970.json', '4140731.json', '2670009.json', '1348373.json', '101994.json', '8633694.json', '17412.json', '1951708.json', '1099091.json', '657941.json', '487651.json', '149958.json', '764944.json', '296722.json', '525406.json', '137185.json', '4189254.json', '1140839.json', '256181.json', '1075993.json', '4641804.json', '8377160.json', '10351139.json', '685832.json', '5363206.json', '1085513.json', '10505339.json', '448297.json', '270876.json', '4294776.json', '837621.json', '11696042.json', '31612.json', '2941858.json', '7436728.json', '1882450.json', '2239557.json', '712615.json', '10027397.json', '8461708.json', '2176092.json', '361484.json', '4082002.json', '1596063.json', '541880.json', '770855.json', '291770.json', '201276.json', '365469.json', '691149.json', '4289492.json', '8504767.json', '9805240.json', '256159.json', '1911793.json', '308712.json', '51459.json', '1567406.json', '758666.json', '1569526.json', '1332492.json', '132918.json', '2583464.json', '2099420.json', '8126516.json', '501221.json', '95621.json', '9877660.json', '1946507.json', '371895.json', '10603379.json', '735131.json', '1828943.json', '96488.json', '2145366.json', '656950.json', '1579151.json', '1415112.json', '1307997.json', '45407.json', '42053.json', '1136153.json', '207673.json', '2061582.json', '11632354.json', '110340.json', '769110.json', '276839.json', '1458260.json', '103983.json', '99024.json', '8444764.json', '1133330.json', '294304.json', '1621263.json', '1211412.json', '71787.json', '764935.json', '1842240.json', '1516386.json', '1187256.json', '2398608.json', '605892.json', '65747.json', '683216.json', '393921.json', '154300.json', '770756.json', '678511.json', '1993830.json', '238832.json', '683277.json', '3265406.json', '1410311.json', '5403145.json', '4376513.json', '500519.json', '721734.json', '2951937.json', '3281001.json', '1756949.json', '1646001.json', '3220562.json', '3364118.json', '2959.json', '651607.json']
# Social Interactions
sub_categories['Social Interactions'] = ['10222.json', '388476.json', '640717.json', '1016144.json', '2726169.json', '801653.json', '132194.json', '955815.json', '978668.json', '53314.json', '1050509.json', '748301.json', '317657.json', '779325.json', '652602.json', '8439489.json', '408788.json', '1508704.json', '1630016.json', '10735351.json', '636488.json', '5718179.json', '885064.json', '18022.json', '9732077.json', '1737487.json', '92884.json', '1403565.json', '90801.json', '712722.json', '5629747.json', '1108035.json', '3202.json', '73940.json', '85793.json', '43230.json', '10652.json', '873798.json', '218320.json', '158584.json', '670293.json', '1863135.json', '2378937.json', '5060231.json', '3529176.json', '792264.json', '11051320.json', '2967763.json', '2080490.json', '1354210.json', '5320968.json', '11329685.json', '16400.json', '1600792.json', '8237069.json', '6653787.json', '1843628.json', '146667.json', '11287035.json', '728937.json', '1352254.json', '505241.json', '1750684.json', '3709458.json', '1385979.json', '229431.json', '1122345.json', '1251055.json', '844761.json', '8274391.json', '52553.json', '87625.json', '10550389.json', '381247.json', '167566.json', '1072592.json', '1852131.json', '388565.json', '240656.json', '1940452.json', '8712867.json', '253884.json', '3633505.json', '730842.json', '1519922.json', '2555150.json', '1147409.json', '101492.json', '1257149.json', '2599284.json', '728530.json', '5744611.json', '879595.json', '10061645.json', '1904245.json', '8586619.json', '10951.json', '146288.json', '937063.json', '8204562.json', '9074401.json', '500615.json', '677287.json', '1291868.json', '89431.json', '178564.json', '1420452.json', '596639.json', '220820.json', '719670.json', '598650.json', '1178612.json', '783239.json', '993630.json', '4521213.json', '1251068.json', '1265472.json', '655381.json', '8204407.json', '389109.json', '2272947.json', '6351593.json', '1513748.json', '8285317.json', '10069279.json', '758956.json', '709426.json', '1934515.json', '876210.json', '661719.json', '5177388.json', '1518702.json', '230251.json', '2722415.json', '2876935.json', '10843890.json', '8383089.json', '9414120.json', '768877.json', '376355.json', '11802073.json', '1163618.json', '5140279.json', '1544581.json', '1824660.json', '68612.json', '55316.json', '2220758.json', '2260146.json', '1866813.json', '4460391.json', '11342115.json', '35628.json', '2251924.json', '490139.json', '11791189.json', '640053.json', '37300.json', '2552067.json', '81626.json', '37707.json', '44252.json', '302491.json', '70170.json', '1382003.json', '4086429.json', '1534139.json', '117623.json', '8535034.json', '5744466.json', '31919.json', '3594099.json', '1604391.json', '726954.json', '537822.json', '3548025.json', '1013320.json', '6030109.json', '3393993.json', '8232847.json', '519032.json', '1115712.json', '38939.json', '2397280.json', '9984299.json', '431600.json', '664594.json', '898313.json', '769528.json', '763575.json', '1166343.json', '6665021.json', '823683.json', '10296762.json', '980747.json', '19973.json', '6220769.json', '217878.json', '1164310.json', '330890.json', '2386187.json', '160615.json', '10439895.json', '4056285.json', '770570.json', '3185933.json', '203417.json', '39805.json', '370867.json', '842540.json', '1041644.json', '1386736.json', '2655111.json', '16203.json', '370133.json', '553166.json', '594041.json', '1432800.json', '4573737.json', '4192009.json', '23751.json', '1492617.json', '10474026.json', '1986208.json', '1508203.json', '829203.json', '152343.json', '269292.json', '1527844.json', '524190.json', '205577.json', '388034.json', '133386.json', '737673.json', '55381.json', '295860.json', '1115418.json', '1989082.json', '10334.json', '10265968.json', '364652.json', '790047.json', '6778483.json', '5838243.json', '803282.json', '19457.json', '11231331.json', '8484733.json', '583674.json', '658230.json', '1311297.json', '2771222.json', '32069.json', '4026562.json', '1943391.json', '24502.json', '461944.json', '15463.json', '8844682.json', '1120813.json', '305268.json', '934251.json', '1493683.json', '8805048.json', '791417.json', '50157.json', '1390769.json', '229815.json', '11655612.json', '4973213.json', '1070289.json', '1761974.json', '178377.json', '202342.json', '725497.json', '1998671.json', '66680.json', '672229.json', '10378.json', '103802.json', '164250.json', '5094448.json', '10952.json', '421012.json', '1517143.json', '990245.json', '1162441.json', '1536188.json', '13981.json', '130837.json', '11116758.json', '63945.json', '890473.json', '1110103.json', '40675.json', '182294.json', '2427134.json', '9635050.json', '1112305.json', '785771.json', '1366401.json', '407428.json', '7658801.json', '1391313.json', '617573.json', '760686.json', '1341230.json', '1424272.json', '542317.json', '34250.json', '10108420.json', '131684.json', '1198614.json', '681158.json', '21269.json', '10795631.json', '281521.json', '51239.json', '9859821.json', '3174994.json', '670618.json', '205463.json', '20907.json', '348812.json', '3754713.json', '164975.json', '63923.json', '18997.json', '773439.json', '796896.json', '2847578.json', '1818663.json', '1893179.json', '855958.json', '1619302.json', '714416.json', '1130711.json', '798426.json', '10177619.json', '3275962.json', '103307.json', '714932.json', '5249546.json', '28286.json', '591793.json', '1103417.json', '67517.json', '1841143.json', '3572424.json', '361964.json', '572607.json', '1248851.json', '9071596.json', '3021566.json', '177210.json', '157576.json', '949438.json', '5382833.json', '4748263.json', '192591.json', '3989411.json', '596979.json', '200339.json', '195469.json', '81839.json', '1683711.json', '3853933.json', '5215585.json', '10913425.json', '920449.json', '306659.json', '8775016.json', '242126.json', '5128294.json', '148712.json', '586411.json', '1413819.json', '55488.json', '5613252.json', '5203353.json', '572755.json', '3746467.json', '551259.json', '938501.json', '4061489.json', '28680.json', '2637205.json', '3949211.json', '232642.json', '126091.json', '7290585.json', '1311229.json', '3026304.json', '120112.json', '8235021.json', '617421.json', '1514159.json', '27996.json', '19082.json', '249766.json', '1851660.json', '6536074.json', '916019.json', '1247707.json', '6644125.json', '296363.json', '9937034.json', '384246.json', '114498.json', '3726505.json', '1815139.json', '109029.json', '6275.json', '407700.json', '333258.json', '8560284.json', '680027.json']
# Cats
sub_categories['Cats'] = ['8643176.json', '1168614.json', '3355955.json', '7513255.json', '695820.json', '5568052.json', '1021221.json', '647705.json', '8559646.json', '8580006.json', '93149.json', '8800019.json', '5510799.json', '8705462.json', '361045.json', '3333629.json', '8769256.json', '8623777.json', '58348.json', '644410.json', '3941589.json', '8673811.json', '5985367.json', '310829.json', '1293019.json', '185626.json', '7191985.json', '1080781.json', '1783954.json', '5025698.json', '8171217.json', '8638013.json', '8587389.json', '1624999.json', '1686551.json', '8635181.json', '8688438.json', '752452.json', '1607754.json', '7379062.json', '8749697.json', '7160524.json', '215338.json', '5292544.json', '37921.json', '1781532.json', '1433813.json', '8562074.json', '8831375.json', '4612384.json', '8690032.json', '8389573.json', '1323910.json', '2796915.json', '6912575.json', '725294.json', '4499069.json', '9490317.json', '1665903.json', '532325.json', '7893.json', '89898.json', '314037.json', '8690199.json', '8805514.json', '21374.json', '6007047.json', '5211993.json', '4246363.json', '8754525.json', '7903.json', '66903.json', '8714045.json', '22036.json', '8831899.json', '158946.json', '596891.json', '971628.json', '7221396.json', '2543703.json', '4634389.json', '8711343.json', '728825.json', '8556286.json', '8636094.json', '698931.json', '8614038.json', '760543.json', '1555284.json', '8661991.json', '8659151.json', '8637867.json', '667672.json', '699068.json', '281723.json', '8730347.json', '1124842.json', '8735515.json', '8657661.json', '71570.json', '513024.json', '5018775.json', '3063223.json', '8680245.json', '1000228.json', '5602949.json', '1648699.json', '8586491.json', '7159516.json', '249318.json', '8735597.json', '5965.json', '8536576.json', '1832419.json', '1276865.json', '6929038.json', '169094.json', '6909318.json', '8733269.json', '8662104.json', '8561680.json', '26192.json', '8779884.json', '229683.json', '393267.json', '1260386.json', '8654601.json', '1439576.json', '1308410.json', '8631717.json', '8661365.json', '1832557.json', '81579.json', '1134873.json', '1725404.json', '1989366.json', '1750590.json', '1443994.json', '8663266.json', '11540610.json', '175614.json', '49904.json', '434194.json', '816919.json', '2403741.json', '1326978.json', '8613851.json', '374191.json', '2100046.json', '1364290.json', '8854746.json', '2373464.json', '2236863.json', '7102282.json', '2361859.json', '8690002.json', '525347.json', '10363271.json', '1548018.json', '60665.json', '25279.json', '8662928.json', '40953.json', '1154252.json', '5478064.json', '8828277.json', '1268542.json', '5381446.json', '2106782.json', '1010540.json', '9319129.json', '8748075.json', '168092.json', '8607306.json', '7931934.json', '8797830.json', '8690202.json', '2603210.json', '213926.json', '950684.json', '1389771.json', '7147888.json', '8585151.json', '6904812.json', '328955.json', '8690146.json', '5392251.json', '7228955.json', '8662840.json', '8663139.json', '3023285.json', '8662047.json', '832358.json', '2243974.json', '8756695.json', '3686252.json', '8796309.json', '8583379.json', '149404.json', '662246.json', '8608504.json', '7097407.json', '480573.json', '6597909.json', '2050777.json', '8766487.json', '103586.json', '2200474.json', '176306.json', '1099417.json', '927873.json', '31630.json', '4259638.json', '761437.json', '1869314.json', '8803242.json', '8752361.json', '1022847.json', '336195.json', '8597635.json', '8613325.json', '968458.json', '630760.json', '1336543.json', '135070.json', '3907.json', '475732.json', '3001252.json', '8570998.json', '11342089.json', '8006386.json', '10175337.json', '8688328.json', '78133.json', '8688514.json', '8695326.json', '717709.json', '8687065.json', '8633196.json', '6365369.json', '44772.json', '1517396.json', '4590617.json', '49551.json', '1354995.json', '5212011.json', '516147.json', '4691356.json', '3306979.json', '1101379.json', '550249.json', '8124.json', '4804.json', '36436.json', '3284.json', '1019893.json', '840654.json', '8748329.json', '30223.json', '8638125.json', '8654336.json', '4475978.json', '8127074.json', '1391968.json', '8684995.json', '58161.json', '1426160.json', '7228595.json', '2387173.json', '387178.json', '8710648.json', '1064584.json', '8684595.json', '2117754.json', '908542.json', '1467342.json', '55987.json', '8687950.json', '8680542.json', '8573180.json', '3490677.json', '1635380.json', '7377.json', '226493.json', '316582.json', '8559068.json', '8661392.json', '8756507.json', '3941333.json', '2129000.json', '8690295.json', '238971.json', '1076428.json', '9391.json', '3798086.json', '3164747.json', '1914067.json', '169706.json', '1797494.json', '7451496.json', '8587400.json', '1545310.json', '5366156.json', '399018.json', '8133519.json', '409519.json', '8714276.json', '8709203.json', '4634587.json', '1047932.json', '8561939.json', '8635059.json', '8661468.json', '5246616.json', '7955621.json', '10177829.json', '4633595.json', '27969.json', '993189.json', '6131065.json', '1102165.json', '8730226.json', '2881750.json', '8782714.json', '11713.json', '1472989.json', '8711717.json', '8636643.json', '2221254.json', '8772452.json', '8586821.json', '1760666.json', '8685537.json', '1855886.json', '848459.json', '3440.json', '7478431.json', '4966683.json', '9288305.json', '1675602.json', '55271.json', '8637678.json', '1646.json', '8586118.json', '9061.json', '5197038.json', '1234274.json', '8670625.json', '8707929.json', '2355471.json', '2944011.json', '8565521.json', '5776889.json', '8579410.json', '888782.json', '1247897.json', '1135354.json', '1040970.json', '7121163.json', '8562191.json', '19187.json', '394494.json', '4652202.json', '70732.json', '8712223.json', '8634835.json', '1029392.json', '714139.json', '8803420.json', '35338.json', '1065540.json', '11726716.json', '393255.json', '7825304.json', '4612133.json', '8654023.json', '8758732.json', '8004364.json', '10904314.json', '5473704.json', '8607504.json', '3327629.json', '285173.json', '1217876.json', '7303889.json', '37692.json', '11112638.json', '8092.json', '802616.json', '8589585.json', '9151254.json', '1421302.json', '377263.json', '2554401.json', '5212406.json', '689026.json', '8715339.json', '152349.json', '11774640.json', '70285.json', '134577.json', '8546968.json', '6974534.json', '146832.json', '8747865.json', '4763467.json', '8604261.json', '168370.json', '7170265.json', '47066.json', '4104107.json', '854754.json', '16878.json', '8690159.json', '8803695.json', '8623781.json', '11717586.json', '1462063.json', '8729282.json', '8636645.json', '8586124.json', '5644022.json', '8635878.json', '112741.json', '8806068.json', '45109.json', '1663912.json', '8806195.json', '5121164.json', '5086023.json', '1204200.json', '11493852.json', '1715998.json', '561616.json', '113347.json', '588360.json', '1647173.json', '4963452.json', '5975604.json', '1519771.json', '17710.json', '8712170.json', '516249.json', '291432.json', '8635618.json', '1194134.json', '8832215.json', '406039.json', '8827851.json', '1377293.json', '8777230.json', '8756097.json', '519743.json', '1213361.json', '4223646.json', '397885.json', '575823.json', '9293187.json', '8685696.json', '8588321.json', '7096834.json', '1451774.json', '10048370.json', '4560665.json', '4794483.json']
# Birds
sub_categories['Birds'] = ['3704322.json', '9108976.json', '25707.json', '8024379.json', '139721.json', '9724156.json', '4405078.json', '135997.json', '1650284.json', '1524578.json', '9078054.json', '804386.json', '9103044.json', '9046038.json', '9332394.json', '9149797.json', '496414.json', '8503603.json', '536916.json', '2884412.json', '1913590.json', '1620061.json', '378854.json', '38569.json', '11642756.json', '3984325.json', '1419353.json', '1598776.json', '6945928.json', '9097680.json', '8404416.json', '138468.json', '1789987.json', '333543.json', '175604.json', '4444780.json', '10257485.json', '9065650.json', '9073696.json', '45977.json', '761165.json', '5710358.json', '3193163.json', '9088734.json', '8049783.json', '9125442.json', '207979.json', '996403.json', '5731628.json', '281410.json', '1723241.json', '1259015.json', '9092211.json', '9123797.json', '5537625.json', '1370688.json', '2312449.json', '8076977.json', '9116050.json', '9788343.json', '52103.json', '1575126.json', '9062399.json', '633150.json', '5039680.json', '9098606.json', '62624.json', '267026.json', '9068852.json', '647712.json', '48324.json', '660344.json', '5566338.json', '176413.json', '1164040.json', '5937736.json', '1290890.json', '9072582.json', '207991.json', '9459638.json', '9099122.json', '3196673.json', '1997786.json', '11633687.json', '9127593.json', '1333783.json', '1272533.json', '1600961.json', '1070843.json', '3192082.json', '704003.json', '840359.json', '4472102.json', '5184861.json', '2262068.json', '804528.json', '1389048.json', '9620498.json', '244444.json', '9786399.json', '9592782.json', '1849351.json', '9322783.json', '9021379.json', '9091698.json', '5829580.json', '9092372.json', '91325.json', '9247919.json', '3064260.json', '9124259.json', '3220788.json', '307555.json', '1794490.json', '100918.json', '9133637.json', '1392668.json', '10524522.json', '10888560.json', '529754.json', '9127603.json', '5563802.json', '9096669.json', '9447522.json', '50860.json', '1329966.json', '1532370.json', '1602639.json', '379604.json', '9071004.json', '3356170.json', '9074045.json', '192411.json', '9122052.json', '9039839.json', '853691.json', '4229305.json', '3768158.json', '342845.json', '1474372.json', '3521168.json', '9059193.json', '668234.json', '149493.json', '9093452.json', '1400422.json', '382136.json', '284174.json', '677698.json', '936406.json', '9127533.json', '5345565.json', '361714.json', '8994929.json', '9777571.json', '145557.json', '5361554.json', '1784186.json', '3572382.json', '5361001.json', '1610489.json', '9098614.json', '10834652.json', '994733.json', '11343116.json', '6793576.json', '9072466.json', '9069553.json', '4397799.json', '9072738.json', '6151214.json', '1246864.json', '33789.json', '1381637.json', '5177581.json', '5935564.json', '1315325.json', '139084.json', '10843759.json', '1316437.json', '802627.json', '3056889.json', '11634950.json', '8544.json', '271827.json', '1578111.json', '9068579.json', '1763743.json', '684927.json', '1280621.json', '10799452.json', '21286.json', '9877135.json', '879067.json', '30680.json', '142778.json', '11679214.json', '9147292.json', '9179505.json', '9742160.json', '405819.json', '29593.json', '49310.json', '1221732.json', '9047133.json', '2577570.json', '6039896.json', '327112.json', '932248.json', '9091700.json', '9058496.json', '2135696.json', '10141679.json', '856355.json', '1057715.json', '9144779.json', '9171899.json', '1303306.json', '9700365.json', '540242.json', '1603763.json', '2102013.json', '5556093.json', '1243482.json', '10606584.json', '9124477.json', '326666.json']
# Traveling with Companions
sub_categories['Traveling with Companions'] = ['7088106.json', '8485486.json', '8714116.json', '8230889.json', '4280662.json', '8298047.json', '11721887.json', '994176.json', '537639.json', '3381629.json', '328758.json', '1194216.json', '8318380.json', '1369224.json', '7995485.json', '1260429.json', '2105779.json', '2145624.json', '1319696.json', '1323379.json', '4139287.json', '687813.json', '7797757.json', '757756.json', '106743.json', '2525884.json', '5665548.json', '1626440.json', '8482161.json', '1542607.json']
# Fun Activities
sub_categories['Fun Activities'] = ['5026.json', '20769.json', '25879.json', '1461729.json', '421979.json', '41630.json', '490372.json', '6010296.json', '722793.json', '22109.json', '3608043.json', '1317262.json', '465719.json', '5913468.json', '145860.json', '6043134.json', '9316691.json', '99709.json', '129014.json', '798468.json', '581568.json', '11664079.json', '562560.json', '480191.json', '864457.json', '1124268.json', '299581.json', '310255.json', '5854673.json', '1954581.json', '160132.json', '195164.json', '3576278.json', '3506316.json', '135129.json', '506477.json', '1887889.json', '1259442.json', '8254079.json', '49528.json', '11779888.json', '453904.json', '3886379.json', '297740.json', '778617.json', '6560963.json', '3887895.json', '223328.json', '103231.json', '4857708.json', '394691.json', '1208800.json', '1790539.json', '3771926.json', '10369083.json', '8932722.json', '1722933.json', '11361.json', '3228140.json', '6082525.json', '925086.json', '270823.json', '1892782.json', '77748.json', '789662.json', '1035707.json', '69275.json', '113744.json', '470623.json', '1855152.json', '683951.json', '2305805.json', '129752.json', '8485380.json', '776133.json', '9299997.json', '3742874.json', '446786.json', '107905.json', '3787.json', '182493.json', '962847.json', '204452.json', '1376132.json', '9866807.json', '289048.json', '325919.json', '617518.json', '1682673.json', '4840076.json', '235528.json', '45504.json', '20886.json', '37586.json', '1307787.json', '54905.json', '194209.json', '1200879.json', '626435.json', '1285916.json', '1726801.json', '271904.json', '11007.json', '1215657.json', '740915.json', '23567.json', '38377.json', '109643.json', '167288.json', '143041.json', '137642.json', '11644767.json', '1012144.json', '194371.json', '1919097.json', '5749355.json', '599842.json', '372903.json', '5835.json', '1530128.json', '43707.json', '127910.json', '1430790.json', '10329695.json', '328008.json', '621660.json', '212806.json', '402098.json', '10240801.json', '11715847.json', '26060.json', '4115486.json', '3477528.json', '677754.json', '551218.json', '128943.json', '105856.json', '9071885.json', '8878010.json', '136984.json', '138563.json', '138826.json']
# Parents
sub_categories['Parents'] = ['148727.json', '8461737.json', '7581.json', '1618336.json', '665403.json', '4330465.json', '1815814.json', '8377586.json', '4526431.json', '2741926.json', '4930668.json', '8507796.json', '8556594.json', '9848405.json', '11051237.json', '8391901.json', '1165192.json', '647900.json', '719262.json', '3804724.json', '4935804.json', '679412.json', '6011573.json', '5983541.json', '3726427.json', '399151.json', '668599.json', '8352422.json', '1590283.json', '461488.json', '3547947.json', '8156039.json', '428824.json', '8291873.json', '8461738.json', '8296206.json', '1387839.json']
# Single Life
sub_categories['Single Life'] = ['1540697.json', '304333.json', '3769560.json', '1039021.json', '859364.json', '41216.json', '599545.json', '1848183.json', '1451706.json', '5385018.json', '1206937.json', '1353033.json', '5820959.json', '42328.json', '1402806.json', '686944.json', '1433707.json', '258445.json', '8295495.json', '1358348.json', '349619.json', '725550.json', '32900.json', '1143205.json', '8183318.json', '29947.json', '1304334.json', '41150.json', '7981784.json', '5599843.json', '383038.json']
# Toys
sub_categories['Toys'] = ['2376660.json', '574250.json', '1333228.json', '290581.json', '481667.json', '170684.json', '9170009.json', '559503.json', '6370945.json', '10177929.json', '8838955.json', '2781171.json', '44496.json', '219385.json', '5313347.json', '2534065.json', '1131996.json', '3765067.json', '1512181.json', '1634265.json', '11363078.json', '5185683.json', '11415641.json', '80386.json', '5868405.json', '6495545.json', '3021809.json', '1158603.json', '206896.json', '3728300.json', '5877784.json', '595689.json', '1339115.json', '2762771.json', '887856.json', '8063530.json', '8777172.json', '880691.json', '5890347.json', '1218497.json', '66012.json', '457211.json', '894959.json', '673654.json', '1123399.json', '7061809.json', '503318.json', '1833335.json', '51686.json', '8595302.json', '64815.json', '7630057.json', '7303996.json', '9628330.json', '125799.json', '8404881.json', '178957.json', '770783.json', '4780497.json', '4746876.json']
# Weddings
sub_categories['Weddings'] = ['430325.json', '8852817.json', '765392.json', '5605825.json', '1331258.json', '1281161.json', '1341009.json', '9704016.json', '8933856.json', '10146392.json', '8258748.json', '1949081.json', '8510063.json', '11230142.json', '10596132.json', '84402.json', '8296137.json', '318995.json', '8518039.json', '3405.json', '8693644.json', '8407036.json', '55605.json', '8910706.json', '8414460.json', '8411025.json', '8446812.json', '666665.json', '13272.json', '2456229.json', '9414388.json', '11694128.json', '1557016.json', '1228398.json', '2247193.json', '8822861.json', '11620183.json', '1487564.json', '8859545.json', '1320443.json', '1667256.json', '11673133.json', '8211371.json', '746495.json', '1651578.json', '154493.json', '1100823.json', '1026632.json', '1874383.json', '11424306.json', '9733841.json', '8296110.json', '2111397.json', '10579535.json', '9542463.json', '103572.json', '1953594.json', '10279932.json', '7122.json', '8232373.json', '1097956.json', '233104.json', '1314701.json', '278013.json', '8363136.json', '1326689.json', '5226222.json', '501443.json', '3069.json', '684884.json', '680412.json', '8455275.json', '1630524.json', '4657422.json', '1273952.json', '2414154.json', '9974937.json', '2622329.json', '8178655.json', '1807375.json', '3436371.json', '8508456.json', '11711984.json', '572206.json', '8890184.json', '172385.json', '8781713.json', '8412342.json', '1155436.json', '1954048.json', '9549942.json', '4237086.json', '8482234.json', '178550.json', '10402730.json', '2700663.json', '5719388.json', '8558045.json', '8411037.json', '8782470.json', '2416599.json', '5367.json', '8684347.json', '8638368.json', '1516174.json', '668215.json', '63355.json', '3890369.json', '1028048.json', '5840602.json', '8910688.json', '1299866.json', '1288774.json', '4075360.json', '10087196.json', '1884588.json', '8252680.json', '8880243.json', '33121.json', '1359758.json', '48479.json', '8433131.json', '2410759.json', '5401378.json', '234538.json', '48464.json', '8276897.json', '653800.json', '1026620.json', '8560354.json', '1953103.json', '8659298.json', '1333504.json', '747812.json', '1203141.json', '10062407.json', '10851671.json', '866536.json', '9536454.json', '1286891.json', '825969.json', '8316992.json', '1806688.json', '107954.json', '137361.json', '985352.json', '1220071.json', '1393437.json', '3005192.json', '8637804.json', '592757.json', '684879.json', '1558115.json', '8312682.json', '8411135.json', '1320061.json', '1199842.json', '934484.json', '24181.json', '134096.json', '8890875.json']
# Neighbors
sub_categories['Neighbors'] = ['553656.json', '1324830.json', '37700.json', '661070.json', '210086.json', '10878745.json', '128655.json', '19025.json', '1813637.json', '8359118.json', '4954545.json', '10357938.json', '131681.json', '134460.json', '259452.json', '10355265.json', '271362.json', '775905.json', '8417618.json', '8528917.json', '126189.json', '1933660.json', '873915.json', '747019.json', '134218.json', '1900686.json', '644848.json', '1707803.json']
# Halloween
sub_categories['Halloween'] = ['3520835.json', '2352743.json', '230650.json', '12643.json', '333806.json', '1080430.json', '1023115.json', '681013.json', '812008.json', '5297880.json', '3777800.json', '227931.json', '44409.json', '686329.json', '1669166.json', '237823.json', '1568626.json', '3350977.json', '1078388.json', '46269.json', '3207003.json', '1645167.json', '10583931.json', '454994.json', '6037065.json', '2302751.json', '9374287.json', '1105908.json', '669578.json', '1645281.json', '5223216.json', '476187.json', '1611171.json', '708559.json', '2135919.json', '2216778.json', '9419142.json', '209042.json', '867247.json', '449581.json', '8146987.json', '1675987.json', '8199157.json', '5270602.json', '2557341.json', '1669009.json', '8333323.json', '3202859.json', '76087.json', '228550.json', '73786.json', '220336.json', '715780.json', '9435452.json', '3524022.json', '3455829.json', '3566292.json', '1483569.json', '648908.json', '1557497.json', '9733767.json', '83117.json', '226148.json', '7602.json', '478934.json', '228235.json', '12358.json', '7019.json', '2401406.json', '3317180.json', '11694408.json', '447742.json', '195203.json', '5367561.json', '76445.json', '707667.json', '1664280.json', '4128287.json', '414927.json', '2321720.json', '452982.json', '40736.json', '219027.json', '2414828.json', '8195214.json', '8204947.json', '8063034.json', '11600245.json', '3548451.json', '1081644.json', '10165.json', '5353320.json']
# Socializing at Work
sub_categories['Socializing at Work'] = ['161348.json', '5058465.json', '1543131.json', '672718.json', '5450353.json', '1895836.json', '5047470.json', '133886.json', '1005466.json', '20349.json', '8804051.json', '778513.json', '1004833.json', '1322919.json', '1902263.json', '8346958.json', '109967.json', '248478.json', '11783722.json']
# Nicknames
sub_categories['Nicknames'] = ['1885627.json', '9049067.json', '47386.json', '655044.json', '1511859.json', '1516430.json']
# Music
sub_categories['Music'] = ['478275.json', '4751298.json', '1489531.json', '93221.json', '1661432.json', '10874272.json', '262634.json', '1412944.json', '596977.json', '26081.json', '700816.json', '387107.json', '5553928.json', '17363.json', '1641600.json', '980626.json', '2697945.json', '98990.json', '1850662.json', '1351463.json', '9228.json', '1633179.json', '1172608.json', '193377.json', '2408490.json', '336872.json', '1398331.json', '11362567.json', '2312999.json', '5931555.json', '1729905.json', '9969230.json', '2788803.json', '1063880.json', '6269870.json', '1254597.json', '200248.json', '3613245.json', '1093011.json', '11495862.json', '748340.json', '11705824.json', '9927888.json', '11763871.json', '1771140.json', '272927.json', '10280412.json', '1480946.json', '1256154.json', '3841612.json', '589483.json', '8786660.json', '251343.json', '280797.json', '506712.json', '1053446.json', '138184.json', '8003114.json', '664356.json', '10988500.json', '7390884.json', '251949.json', '6702.json', '52509.json', '1726853.json', '1438662.json', '2143879.json', '1946964.json', '2172298.json', '273068.json', '140073.json', '1726042.json', '285096.json', '172944.json', '1540834.json', '201117.json', '1402476.json', '94706.json', '34435.json', '359237.json', '842968.json', '460274.json', '8704068.json', '676994.json', '9031332.json', '4320350.json', '29483.json', '1405921.json', '1635259.json', '73805.json', '182415.json', '1741796.json', '1121122.json', '31699.json', '9367546.json', '6245682.json', '88363.json', '1638979.json', '1383770.json', '327096.json', '556199.json', '257449.json', '242132.json', '493162.json', '45648.json', '1333229.json', '1518070.json', '1596211.json', '1346903.json', '7327622.json', '94253.json', '5436546.json', '2171708.json', '7349482.json', '1187297.json', '26327.json', '2014585.json', '10861010.json', '4962892.json', '384570.json', '746892.json', '654221.json', '98374.json', '176793.json', '458637.json', '948457.json', '345844.json', '1671350.json', '130848.json', '150619.json', '603505.json', '4585607.json', '94394.json', '2525185.json', '657572.json', '1678425.json', '8803550.json', '7333421.json', '8967820.json', '2146857.json', '11155884.json', '12642.json', '246809.json', '669924.json', '10274653.json', '699711.json', '328046.json', '391154.json', '164720.json', '11261442.json', '33996.json', '9869407.json', '124305.json', '7333401.json', '14470.json', '1767216.json', '9654766.json', '43694.json', '22387.json', '132947.json', '7056542.json', '531143.json', '793603.json', '5345976.json', '191737.json', '1502719.json', '6948141.json', '198392.json', '9714308.json', '9636662.json', '333923.json', '9763409.json', '490884.json', '67758.json', '5955117.json', '412306.json', '1529609.json', '1288269.json', '3207691.json', '1041708.json', '127944.json', '8962395.json', '848514.json', '223397.json', '523328.json', '5722120.json', '448011.json', '82124.json', '12588.json', '9881536.json', '368628.json', '10914977.json', '17152.json', '110041.json', '704808.json', '22070.json', '3418427.json', '1455600.json', '11440289.json', '2118354.json', '2328590.json', '430285.json', '694942.json', '1939983.json', '60254.json', '245625.json', '167644.json', '1190938.json', '528191.json', '1980184.json', '228895.json', '10315290.json', '789045.json', '582752.json', '998461.json', '540861.json', '1305977.json', '489987.json', '1485719.json', '1395067.json', '11488431.json', '2463005.json', '286852.json', '152822.json', '4238054.json', '381237.json', '2505365.json', '11403.json', '67905.json', '1358263.json', '10847037.json', '1391570.json', '1313948.json', '29169.json', '516411.json', '11418176.json', '2527828.json', '235726.json', '44265.json', '239181.json', '3029.json', '688808.json', '548246.json', '6063897.json', '516295.json', '4548740.json', '386360.json', '1675953.json', '2516764.json', '9707722.json', '889558.json', '153993.json', '652110.json', '10808505.json', '1335335.json', '11577369.json', '3333286.json', '1953557.json', '7330744.json', '76699.json', '1609701.json', '132757.json', '2799845.json', '3484335.json', '748917.json', '800947.json', '642917.json', '51513.json', '1536902.json', '351271.json', '30329.json', '2414909.json', '6465073.json', '2808347.json', '4352225.json', '649033.json', '289400.json', '1134890.json', '4727863.json', '1971918.json', '1613859.json', '250811.json', '41991.json', '4329317.json', '3844904.json', '8132848.json', '21955.json', '155257.json', '282626.json', '402911.json', '5903571.json', '256480.json', '220656.json', '632004.json', '10615528.json', '682788.json', '1175723.json', '554907.json', '1610018.json', '539164.json', '7899.json', '11644534.json', '1542140.json', '6877931.json', '10952320.json', '7294405.json', '1860601.json', '417196.json', '11228082.json', '1875100.json', '271926.json', '49159.json', '17098.json', '1177071.json', '5478474.json', '47813.json', '1261666.json', '1590418.json', '9346.json', '4279328.json', '1539822.json', '3508162.json', '689516.json', '138176.json', '531885.json', '1304185.json', '10821151.json', '23441.json', '3763936.json', '2849021.json', '830109.json', '328650.json', '22180.json', '176672.json', '215427.json', '785138.json', '333362.json', '2420954.json', '7295687.json', '656152.json', '9506340.json', '7351763.json', '2808632.json', '3571165.json', '1042552.json', '1877127.json', '7327448.json', '2647504.json', '317308.json', '1819811.json', '10155106.json', '3263958.json', '2440699.json', '1539382.json', '3710.json', '11369926.json', '697251.json', '632049.json', '10598604.json', '8635145.json', '1871548.json', '7317683.json', '118221.json', '351529.json', '1905906.json', '10546.json', '258041.json', '6166077.json', '3332208.json', '4554.json', '10703878.json', '4277480.json', '1959116.json', '2975723.json', '1361924.json', '721585.json', '664644.json', '553562.json', '3196609.json', '211983.json', '1897164.json', '417289.json', '4733905.json', '5730681.json', '3147838.json', '260878.json', '485474.json', '32708.json', '1383560.json', '1050350.json', '4712689.json', '6083141.json', '9654391.json', '804530.json', '566253.json', '1512706.json', '457512.json', '2294962.json', '11575819.json', '177447.json', '4447628.json', '161710.json', '521330.json', '308154.json', '4046987.json', '1257063.json', '3343.json', '657633.json', '5501.json', '345181.json', '77638.json', '10832861.json', '207614.json', '136409.json', '442478.json', '1048218.json', '11610976.json', '4150.json', '60702.json', '2409636.json', '1181556.json', '10942179.json', '3945.json', '5402533.json', '4576807.json', '28310.json', '6830975.json', '10946182.json', '7314346.json', '9095528.json', '18821.json', '180912.json', '2241116.json', '1812225.json', '465662.json', '138342.json', '321604.json', '503955.json', '720728.json', '91185.json', '583679.json', '10791269.json', '454596.json', '6196324.json', '778443.json', '34372.json', '331096.json', '96181.json', '2568.json', '8830409.json', '1339174.json', '10335059.json', '83415.json', '173263.json', '769767.json', '5611730.json', '7474649.json', '7314370.json', '113970.json', '33993.json', '2095727.json', '6434678.json', '10769929.json', '513758.json', '99491.json', '84759.json', '2716670.json', '2239983.json', '1371696.json', '266976.json', '1010850.json', '98003.json', '9675252.json', '256882.json', '249274.json', '10136489.json', '1681033.json', '53445.json', '194770.json', '7349448.json', '4399289.json', '11789083.json', '927780.json', '9094362.json', '3766633.json', '9809106.json', '268440.json', '41141.json', '1397214.json', '6287079.json', '1700586.json', '1853230.json', '45682.json', '230947.json', '4919546.json', '510699.json', '18568.json', '3898168.json', '313692.json', '175289.json', '5667122.json', '657605.json', '38536.json', '756232.json', '2587542.json', '5842721.json', '417907.json', '105470.json', '533286.json', '6971750.json', '131482.json', '416707.json', '664060.json', '940311.json', '11427987.json', '831075.json', '10974912.json', '9710699.json', '1644411.json', '422130.json', '9837110.json', '2574069.json', '7327539.json', '56797.json', '1181222.json', '1542482.json', '2855041.json', '236703.json', '617974.json', '944391.json', '7352012.json', '4126.json', '110023.json', '2959052.json', '985677.json', '1007156.json', '11365663.json', '11214911.json', '1089858.json', '97066.json', '279944.json', '118151.json', '4464.json', '196625.json', '1222197.json', '1169190.json', '3386398.json', '381941.json', '243140.json', '873512.json', '178357.json', '1211989.json', '2669726.json', '41603.json', '1505471.json', '9800824.json', '3394171.json', '49301.json', '45322.json', '1471425.json', '81995.json', '2004368.json', '138181.json', '841785.json', '6801803.json', '142661.json', '7085551.json', '139817.json', '59213.json', '1329964.json', '1012948.json', '9916562.json', '796521.json', '749523.json', '3794046.json', '164704.json', '616650.json', '6544697.json', '725478.json', '21864.json', '316576.json', '8963.json', '1683613.json', '521833.json', '582478.json', '12715.json', '1693097.json', '165338.json', '386924.json', '179462.json', '296596.json', '1133120.json', '5481862.json', '1179333.json', '5325353.json', '1797420.json', '713321.json', '1181557.json', '181713.json', '1129690.json', '136664.json', '173151.json', '1334403.json', '3642078.json', '10355225.json', '663414.json', '5916.json', '1536976.json', '2214566.json', '2132952.json', '700472.json', '192040.json', '581901.json', '11767929.json', '2191519.json', '1610295.json', '352604.json', '71485.json', '9062981.json', '7331033.json', '657802.json', '645096.json', '876310.json', '390253.json', '59408.json', '28818.json', '10847245.json', '49263.json', '289594.json', '2667.json', '659774.json', '415784.json', '703255.json', '1067654.json', '2421550.json', '5361745.json', '7306536.json', '196730.json', '651539.json', '2472759.json', '2901294.json', '625414.json', '4749595.json', '867995.json', '8969348.json', '1492943.json', '28033.json', '199797.json', '11563441.json', '7512329.json', '10954623.json', '219856.json', '158949.json', '1065887.json', '284148.json', '631894.json', '49977.json', '769937.json', '1657040.json', '4589778.json', '67126.json', '79568.json', '7333707.json', '8041410.json', '121128.json', '11751241.json', '1730618.json', '448291.json', '10191.json', '681457.json', '2157697.json', '11657203.json', '9951241.json', '595338.json', '518519.json', '2065715.json', '558429.json', '174282.json', '9693988.json', '56749.json', '53398.json', '1383549.json', '766135.json', '59280.json', '292004.json', '737340.json', '10322466.json', '175939.json', '9877.json', '686150.json', '398911.json', '1117638.json', '4117603.json', '149293.json', '1246727.json', '82238.json', '1016170.json', '4577857.json', '72607.json', '1107815.json', '14828.json', '10615537.json', '597942.json', '2136490.json', '1014136.json', '2212525.json', '178761.json', '14445.json', '519735.json', '398481.json', '733006.json', '647025.json', '79437.json', '10097650.json', '644062.json', '11364669.json', '2839853.json', '708301.json', '10502.json', '4563592.json', '864474.json', '10888549.json', '578267.json', '1349732.json', '10965888.json', '666447.json', '37919.json', '351111.json', '1374918.json', '4535825.json', '11624306.json', '189919.json', '7333423.json', '666446.json', '37277.json', '463436.json', '2378671.json', '83279.json', '891136.json', '651298.json', '631899.json', '7022375.json', '1879003.json', '166050.json', '6913977.json', '67613.json', '30232.json', '9091612.json', '1259040.json', '101356.json', '2125252.json', '170350.json', '19276.json', '2145717.json', '936698.json', '696940.json', '6459494.json', '452582.json', '4170800.json', '1158852.json', '504716.json', '291629.json', '1418890.json', '105513.json', '9769362.json', '479017.json', '1246419.json', '1652234.json', '116850.json', '19427.json', '271345.json', '72169.json', '1421303.json', '7314394.json', '148704.json', '4734303.json', '2657259.json', '1350375.json', '403738.json', '29320.json', '213148.json', '161449.json', '716241.json', '364710.json', '1903531.json', '6878068.json', '3132567.json', '43747.json', '7008507.json', '691319.json', '3191097.json', '8905654.json', '1987023.json', '806642.json', '1608857.json', '631092.json', '11376874.json', '19446.json', '1471593.json', '134297.json', '6570497.json', '885513.json', '1383339.json', '450858.json', '11254061.json', '662063.json', '10932749.json', '1084554.json', '1259060.json', '30642.json', '8613878.json', '567672.json', '5564459.json', '1490945.json', '5917167.json', '9693897.json', '459946.json', '1454063.json', '655078.json', '887241.json', '1577318.json', '2009602.json', '1315968.json', '1456232.json', '4296448.json']
# Dating
sub_categories['Dating'] = ['527106.json', '1551481.json', '8917105.json', '7478183.json', '1149519.json', '4433721.json', '1039980.json', '681475.json', '7220.json', '36202.json', '15562.json', '4528608.json', '149966.json', '26994.json', '173932.json', '597018.json', '1761438.json', '8002031.json', '877169.json', '804693.json', '494501.json', '25253.json', '895843.json', '836326.json', '9574629.json', '587258.json', '1765379.json', '11129854.json', '3070727.json', '668617.json', '1180549.json', '26107.json', '3983340.json', '2660447.json', '944551.json', '9439571.json', '28725.json', '1660521.json', '351867.json', '972526.json', '5034338.json', '8915564.json', '830155.json', '1080934.json', '94157.json', '208072.json', '4649529.json', '132589.json', '2303557.json', '148402.json', '11468.json', '21031.json', '4713233.json', '8073.json', '103329.json', '42509.json', '592979.json', '5451381.json', '1388940.json', '594792.json', '195539.json', '106243.json', '16246.json', '1035868.json', '930489.json', '39483.json', '214987.json', '6846.json', '208418.json', '321944.json', '33098.json', '1936534.json', '1603404.json', '9095812.json', '485776.json', '252308.json', '562869.json', '1061370.json', '15112.json', '322295.json', '1905983.json', '14997.json', '2741560.json', '788844.json', '8430125.json', '11094778.json', '680029.json', '1488090.json', '7451277.json', '1640899.json', '1182484.json', '229539.json', '36018.json', '759419.json', '886370.json', '1276042.json', '594654.json', '5783834.json', '49000.json', '5683890.json', '1140830.json', '38440.json', '597594.json', '1542275.json', '208419.json', '456193.json', '736694.json', '1750743.json', '5380.json', '536632.json', '320179.json', '249768.json', '875933.json', '1824596.json', '1874388.json', '59485.json', '1299589.json', '586983.json', '95478.json', '749862.json', '1065922.json', '160240.json', '1106682.json', '6437876.json', '495126.json', '139940.json', '50866.json', '127698.json', '204841.json', '1846607.json', '504329.json', '372275.json', '1122148.json', '794261.json', '1625409.json', '1248307.json', '1064221.json', '2157895.json', '494799.json', '40431.json', '50061.json', '764432.json', '224720.json', '1792519.json', '1680216.json', '192336.json', '6627.json', '39924.json', '30713.json', '38773.json', '145929.json', '634302.json', '647143.json', '934756.json', '22692.json', '2687250.json', '9103655.json', '435262.json', '599247.json', '394069.json', '1877245.json', '248030.json', '79697.json', '1763701.json', '31790.json', '8736114.json', '1255780.json', '1764928.json', '42046.json', '140091.json', '1700192.json', '652947.json', '2080326.json', '824674.json', '11342353.json', '2574128.json', '10362575.json', '5127790.json', '1282058.json', '3295715.json', '257886.json', '770200.json', '2184380.json', '136986.json', '722120.json', '399548.json', '539725.json', '1243510.json', '2575810.json', '28163.json', '10417825.json', '1662078.json', '1175186.json', '2776053.json', '1477634.json', '82590.json', '10799460.json', '703949.json', '5939293.json', '447642.json', '89734.json', '5752442.json', '59586.json', '3880297.json', '193845.json', '838522.json', '26971.json', '50836.json', '43060.json', '1142430.json', '1170054.json', '2043818.json', '1321897.json', '42935.json', '1053354.json', '9496817.json', '1614806.json', '1968959.json', '159805.json', '386399.json', '2130700.json', '8618952.json', '5931334.json', '515417.json', '10360892.json', '2053.json', '352579.json', '554320.json', '1195653.json', '80610.json', '10224.json', '1227206.json', '8260245.json', '690777.json', '1777006.json', '1222815.json', '18643.json', '576621.json', '1257850.json', '7375.json', '517148.json', '470747.json', '7660.json', '5914518.json', '927805.json', '2657837.json', '1007483.json', '200665.json', '8709040.json', '140184.json', '729511.json', '10422941.json', '12599.json', '190418.json', '745958.json', '1267398.json', '1188000.json', '2135488.json', '227404.json', '596967.json', '89903.json', '5357448.json', '9096263.json', '4729742.json', '504461.json', '371716.json', '815417.json', '398626.json', '27301.json', '703882.json', '633205.json', '5630.json', '3684645.json', '6509.json', '9514215.json', '1827558.json', '551179.json', '934317.json', '429207.json', '9912017.json', '4665572.json', '379808.json', '217933.json', '1331530.json', '1411302.json', '33494.json', '1156805.json', '47106.json', '1151769.json', '1097628.json', '700997.json', '187470.json', '1158289.json', '44141.json', '189481.json', '427920.json', '1237801.json', '863793.json', '2310219.json', '197036.json', '273448.json', '1894050.json', '108016.json', '9741.json', '979681.json', '5932391.json', '4669116.json', '666136.json', '11072.json', '280335.json', '324262.json', '243206.json', '724637.json', '38574.json', '9071360.json', '893545.json', '33142.json', '170876.json', '749604.json', '11344043.json', '9004299.json', '827859.json', '8684328.json', '1063805.json', '1142217.json', '2330396.json', '984459.json', '32587.json', '1081292.json', '229231.json', '49817.json', '9072080.json', '1140031.json', '7712.json', '1869302.json', '1963894.json', '991440.json', '1879302.json', '227872.json', '499752.json', '10371447.json', '1291869.json', '31734.json', '15512.json', '8627085.json', '183700.json', '735397.json', '199927.json', '1387688.json', '38942.json', '41027.json', '65578.json', '2134596.json', '449667.json', '1355684.json', '720443.json', '82354.json', '4280845.json', '9503949.json', '602257.json', '10527713.json', '9048451.json', '2189404.json', '176054.json', '148393.json', '430007.json', '3633945.json', '766327.json', '1697994.json', '8532895.json', '447706.json', '272280.json', '2527043.json', '6041709.json', '1860508.json', '6256679.json', '49547.json', '98222.json', '64055.json', '687643.json', '228792.json', '750778.json', '3042832.json', '29612.json', '219348.json', '361718.json', '355181.json', '8469509.json', '36119.json', '784874.json', '10302.json', '494509.json', '547830.json', '2253605.json', '702248.json', '3630501.json', '762371.json', '909689.json', '42274.json', '260705.json', '3568238.json', '3146857.json', '1173011.json', '80901.json', '1032940.json', '136825.json', '131899.json', '202355.json', '1376831.json', '1844668.json', '545724.json', '692590.json', '1813475.json', '5868.json', '27509.json', '1690771.json', '799163.json', '24217.json', '443859.json', '766304.json', '750666.json', '61349.json', '1120107.json', '1543928.json', '1012528.json', '1671.json', '5380756.json', '1940343.json', '120640.json', '6036178.json', '129031.json', '2948615.json', '1185923.json', '8915.json', '8587641.json', '611321.json', '1481754.json', '1323324.json', '3753910.json', '5471857.json', '10799478.json', '13332.json', '1679102.json', '517330.json', '503236.json', '127107.json', '9602928.json', '620780.json', '13774.json', '1522113.json', '8608065.json', '688484.json', '947928.json', '726075.json', '314290.json', '1667330.json', '6434.json', '1129833.json', '1040058.json', '8175338.json', '1322762.json', '8123108.json', '662264.json', '17809.json', '9688988.json', '3354896.json', '8586641.json', '1140005.json', '1274276.json', '3896073.json', '1896805.json', '207398.json', '656078.json', '1064244.json', '11623299.json', '1194688.json', '790139.json', '9089603.json', '10064960.json', '2866561.json', '690826.json', '716050.json', '9477118.json', '1322739.json', '1589965.json', '9071006.json', '11579378.json', '138882.json', '2177877.json', '1699064.json', '1024745.json', '3489271.json', '5541859.json', '509159.json', '2551669.json', '47831.json', '4853139.json', '18965.json', '300949.json', '1371967.json', '1284700.json', '122826.json', '421442.json', '1288867.json', '1357654.json', '3895.json', '3543145.json', '3649771.json', '4080822.json', '851212.json', '188288.json', '125859.json', '288232.json', '1573605.json', '4181110.json', '95850.json', '728677.json', '16279.json', '854114.json', '182555.json', '8827924.json', '1597238.json', '894037.json', '742590.json', '682440.json', '47364.json', '1604398.json', '1541944.json', '9127348.json', '1284766.json', '4209039.json', '1308967.json', '1278680.json', '492394.json', '6274.json', '155300.json', '230486.json', '679507.json', '83999.json', '594421.json', '3769915.json', '10251363.json', '1003329.json', '594708.json', '1037363.json', '161425.json', '1807847.json', '604826.json', '1004141.json', '428957.json', '123314.json', '6855469.json', '24856.json', '56817.json', '596522.json', '25656.json', '1248308.json', '714768.json', '662776.json', '407238.json', '615077.json', '1328041.json', '3989299.json', '1179904.json', '1268643.json', '2117154.json', '1133428.json', '918483.json', '183251.json', '4127974.json', '1812649.json', '5572492.json', '10222657.json', '1727112.json', '814561.json', '432228.json', '1662050.json', '2081781.json', '8789036.json', '13206.json', '2096394.json', '883537.json', '2154279.json', '375474.json', '1460659.json', '178937.json', '1182429.json', '73503.json', '3892.json', '1675890.json', '612340.json', '2651.json', '688924.json', '1554766.json', '1347940.json', '132175.json', '689118.json', '1181157.json', '497548.json', '1342187.json', '843984.json', '8514222.json', '2194419.json', '4719846.json', '1472824.json', '1690645.json', '4693117.json', '4202879.json', '1558569.json', '5713223.json', '48900.json', '214394.json', '680545.json', '2457532.json', '1445255.json', '19775.json', '895088.json', '1449299.json', '4023639.json', '153758.json', '28643.json', '1007473.json', '694414.json', '1134421.json', '11661609.json', '1120874.json', '2433331.json', '198575.json', '3729392.json', '1077767.json', '67214.json', '11754.json', '515890.json', '4775273.json', '5102891.json', '533275.json', '143430.json', '2534190.json', '1140648.json', '83994.json', '82813.json', '62344.json', '278719.json', '1263741.json', '2966609.json', '593886.json', '4684893.json', '837650.json', '1078806.json', '12400.json', '1004418.json', '501851.json', '12791.json', '2111449.json', '1028873.json', '1808183.json', '45563.json', '4655122.json', '348960.json', '345276.json', '266048.json', '686537.json', '1120747.json', '26128.json', '153187.json', '1003596.json', '74588.json', '1152481.json', '173931.json', '376804.json', '11343517.json', '354099.json', '1756524.json', '31182.json', '1762217.json', '71289.json', '3076664.json', '19295.json', '9045545.json', '1800116.json', '1940893.json', '1138121.json', '854276.json', '4858929.json', '737176.json', '71134.json', '829068.json', '908142.json', '541306.json', '5074840.json', '782905.json', '5987573.json', '752871.json', '139165.json', '728016.json', '808631.json', '1230801.json', '7233809.json']


In [0]:
for k in sub_categories.keys():
  print(k)

In [35]:
total_files = 0
for k in sub_categories.keys():
  total_files += len(sub_categories[k])
print(total_files)

11226


Once imports and functions are done, upload a JSON file, get its name and parse it

# Gameplay functions

In [19]:
uploaded = files.upload()

Saving 3029.json to 3029.json


In [20]:
try:
  file_name = list(uploaded.keys())[0]
  print(file_name)
except:
  f = open('/content/3029.json')
  file_name = f.name.split('/')
  file_name = file_name[len(file_name) - 1]

3029.json


In [0]:
def trim_beginning_punctuation(sentence):
  to_return = ''
  letter_seen = False
  for c in sentence:
    if c in string.punctuation or c in string.whitespace:
      if letter_seen:
        to_return += c
    else:
      if c in string.ascii_letters or c in string.digits:
        if not letter_seen:
          letter_seen = True
        to_return += c
  return to_return

def milestone_1_results(filename, include_desc):
  # include_desc -> Do you want to include imperative description sentences as game steps?
  steps, descriptions, method_names, title = get_article(filename)
  # print the title of the article
  #print('Title: ' + title)
  print(title)
  # Once the necessary fields are filled, find imperatives
  imperatives = []

  for step in steps:
    # First split the headline into sentences
    headline_sentences = step[1].split('.')
    if len(headline_sentences) == 1:
      headline_sentences = headline_sentences[0].split(':')
    for headline in headline_sentences:
      trimmed_headline = trim_beginning_punctuation(headline)
      trimmed_headline = trimmed_headline.replace('\n', '')
      if len(trimmed_headline) == 0:
          continue
      if imperative_check(trimmed_headline):
        if trimmed_headline not in imperatives:
          imperatives.append(trimmed_headline)
        break # No two imperatives from the same headline are added.
    if include_desc:
      description = descriptions[step[1]]
      # Split the description paragraph into sentences
      sentences = description.split('.')
      for sentence in sentences:
        trimmed_sentence = trim_beginning_punctuation(sentence)
        trimmed_sentence = trimmed_sentence.replace('\n', '')
        if len(trimmed_sentence) == 0:
          continue
        if imperative_check(trimmed_sentence):
          if trimmed_sentence not in imperatives:
            imperatives.append(trimmed_sentence)
          break # No two imperatives from the same description are added. !!! REMOVE THIS IF you want multiple imperatives
    
  #third_person = [changePerson(imperative) for imperative in imperatives]
  return imperatives, title, descriptions

Methods for choice picking

In [0]:
def pick_choices(current_step, commands):
  if len(commands) - 1 == current_step:
    return None, None
  elif len(commands) - 2 == current_step:
    # Can only pick one wrong choice
    return len(commands) - 1, None
  else:
    # pick 2 that are not in steps_seen
    options_first = [num for num in range(current_step + 1, len(commands))]
    first = random.choice(options_first)
    options_second = [num for num in range(current_step + 1, len(commands)) if num != first]
    second = random.choice(options_second)
    return first,second

def pick_choices_alt(title, n, similars, curr_step, further_steps):
  # first filter the ones that are too similar to actual correct step
  user_sent = construct_sentence_vector(curr_step)
  s = [sim for sim in similars if vectors.similarity(construct_sentence_vector(sim), user_sent) < 0.9 and vectors.similarity(construct_sentence_vector(sim), user_sent) > 0.1]
  s = s + further_steps
  first = random.choice(s)
  s_2 = [step for step in s if step != first]
  second = random.choice(s_2)
  return first,second

Start extracting from article here:


In [0]:
# Send True to include descriptions as well!
known_commands, title, descriptions = milestone_1_results(file_name, False)

How to Fart in Class Without Anyone Noticing


In [0]:
try:
  embeddings = model.encode(known_commands, bsize=128, tokenize=False, verbose=True)
  print('nb sentences encoded : {0}'.format(len(embeddings)))
except:
  print('This article has no imperatives!')

Nb words kept : 65/72 (90.3%)
Speed : 7.5 sentences/s (cpu mode, bsize=128)
nb sentences encoded : 5


Wrong choice warnings: pick one of these if the player picks a wrong option

In [0]:
wrong_warning = ['Whoa! You should not do that yet.', 'Umm.. maybe you should not do that now.', 'That could work in an ideal world, but that is not what we have here. Try again please!', 'No no no...']

Start the game here:

In [97]:
current_step = 0
steps_seen = []
end_game = False
file_name = ''
# First get the game
print('Welcome to WikiHow Dungeon!')
print('You can either pick a category to play a game from or you can upload your own game.')
print('a) Pick a category')
print('b) Upload JSON')
picked = ''
is_picked = False
while not is_picked:
  picked = input('>')
  if picked.lower() == 'a' or picked.lower() == 'b':
    is_picked = True
  else:
    print('Try again.')
# If from category
if picked.lower() == 'a':
  print('Enter the corresponding letter for a category below:')
  print('a) School: university, other school stuff')
  print('b) Youth: dating, culture, interactions, social events')
  print('c) Work: going to work, socializing at work')
  print('d) Animals: dogs, cats, birds')
  print('e) Fun: activities, parties, toys, music, drinks, games, halloween')
  print('f) Home and Family: parents, weddings, neighbors')
  print('g) Pop Culture: fandom, celebrities')
  print('h) Relationships: single life, dating, relationship issues')
  print('i) Friendship: friends, traveling with companions, nicknames')
  print('k) Appearance: fashion, looking good')
  is_picked = False
  while not is_picked:
    picked = input('>')
    if len(picked) == 1 and picked.lower() in 'abcdefghik':
      is_picked = True
    else:
      print('Try again.')
  categories = []
  if picked.lower() == 'a':
    categories = ['University', 'School Stuff']
  elif picked.lower() == 'b':
    categories = ['Youth Dating', 'Youth Culture', 'Social Interactions for Youth', 'Social Events for Youth']
  elif picked.lower() == 'c':
    categories = ['Job Attendance', 'Socializing at Work', 'School Stuff']
  elif picked.lower() == 'd':
    categories = ['Dogs', 'Cats', 'Birds']
  elif picked.lower() == 'e':
    categories = ['Drinks', 'Games', 'Parties', 'Fun Activities', 'Toys', 'Halloween', 'Music']
  elif picked.lower() == 'f':
    categories = ['Youth and Family', 'Parents', 'School Stuff', 'Weddings', 'Neighbors']
  elif picked.lower() == 'g':
    categories = ['Celebrities', 'Fandom']
  elif picked.lower() == 'h':
    categories = ['Relationship Issues', 'Single Life', 'Dating']
  elif picked.lower() == 'i':
    categories = ['Friends', 'Social Interactions', 'Traveling with Companions', 'Nicknames']
  elif picked.lower() == 'k':
    categories = ['Fashion', 'Looking Good']
  c = random.choice(categories)
  articles = sub_categories[c]
  file_name = random.choice(articles)
  file_name = "/content/drive/Shared drives/CIS 700 Project/category_files/" + file_name

elif picked.lower() == 'b':
  # Upload a file
  uploaded = files.upload()
  try:
    file_name = list(uploaded.keys())[0]
  except:
    print('File not uploaded, will play the default file.')
    f = open('/content/3029.json')
    file_name = f.name.split('/')
    file_name = file_name[len(file_name) - 1]

# Get the commands, descriptions and title
known_commands, title, descriptions = milestone_1_results(file_name, False)
print(known_commands)
print(descriptions)
try:
  embeddings = model.encode(known_commands, bsize=128, tokenize=False, verbose=True)
  #print('nb sentences encoded : {0}'.format(len(embeddings)))
except:
  print('This article has no imperatives!')

print('Game: ' + title + '\n')
get_description_title(title)
similars = find_steps_similar_articles(title, 1)
while not end_game:
  if current_step == len(known_commands):
    end_game = True
    print('You reached the end of the game')
  else:
    if current_step == 0:
      print('What do you do first?')
    else:
      print('What do you do next?')
    print('You:')
    #first, second = pick_choices(current_step, known_commands)
    choices = []
    correct_choice = known_commands[current_step] # this is the correct choice
    further_steps = [known_commands[i] for i in range(current_step + 1, len(known_commands))]
    f, s = pick_choices_alt(title, 3, similars, correct_choice, further_steps)
    choices.append(f)
    choices.append(s)
    choices.append(correct_choice)
    random.shuffle(choices) # shuffle the order of choices
    for choice in choices:
      print('\t' + choice.lower())
    correct_found = False
    while not correct_found:
      user_pick = input('>')
      if user_pick.lower() == 'quit':
        correct_found = True
        end_game = True
        print('You ended the game.')
        continue
      # Check if what the user picked is correct
      most_similar = find_similar_command(user_pick, [known_commands[i] for i in range(current_step, len(known_commands))])
      if most_similar.lower() == correct_choice.lower():
        # Correct!
        current_step += 1
        correct_found = True
        # Get description
        print(get_step_description_game(correct_choice, descriptions))
      else:
        print(random.choice(wrong_warning))


Welcome to WikiHow Dungeon!
You can either pick a category to play a game from or you can upload your own game.
a) Pick a category
b) Upload JSON
>a
Enter the corresponding letter for a category below:
a) School: university, other school stuff
b) Youth: dating, culture, interactions, social events
c) Work: going to work, socializing at work
d) Animals: dogs, cats, birds
e) Fun: activities, parties, toys, music, drinks, games, halloween
f) Home and Family: parents, weddings, neighbors
g) Pop Culture: fandom, celebrities
h) Relationships: single life, dating, relationship issues
i) Friendship: friends, traveling with companions, nicknames
k) Appearance: fashion, looking good
>b
How to Accept that Your Crush Doesn't Like You
['Spend time on your hobbies', 'Meet new people', 'Try something that is completely different', 'Respect your crushs boundaries', 'Explain that you need some time to recover', 'Stay away from places you think your crush may be', 'Avoid asking your friends about your c